In [2]:
pip install --upgrade --user stata_setup

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for stata_setup: filename=stata_setup-0.1.3-py3-none-any.whl size=6636 sha256=4ce23b051b4639a9b0cab24b879f692fa7e6823111cd4af65251a1b45016e834
  Stored in directory: c:\users\zach\appdata\local\pip\cache\wheels\26\09\30\e32fe4864266643a13b56943f27403e22775694610a3fb6f9f
Successfully built stata_setup
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install pystata

  Obtaining dependency information for pystata from https://files.pythonhosted.org/packages/d7/98/0f53a1fea2eb07418465f534e536cc9167d79313261f3a8badcb3538059d/pystata-0.0.1-py3-none-any.whl.metadata
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

# Load Stata file directly
df = pd.read_stata(r"C:\Users\Zach\PycharmProjects\AppliedML\ces_gdp_rct_v1a.dta")

# Check the first few rows
print(df.head())

# Then you can run rreg2_py(df, ...) directly


  country             pr2010 pr2110   age  \
0                        NaN    NaN   NaN   
1      BE  Non-probabilistic   CAWI  54.0   
2      BE  Non-probabilistic   CAWI  54.0   
3      BE  Non-probabilistic   CAWI  54.0   
4      BE  Non-probabilistic   CAWI  54.0   

                                      b2010              b3300    b7010  \
0                                       NaN                NaN      NaN   
1  Have no job but would like to have a job  Rented house/flat  14000.0   
2  Have no job but would like to have a job  Rented house/flat  14000.0   
3  Have no job but would like to have a job  Rented house/flat  14000.0   
4  Have no job but would like to have a job  Rented house/flat  14000.0   

                                       q2021  wave          j1020  ...  \
0                                        NaN     7            NaN  ...   
1  Unemployed and actively looking for a job     8  Health sector  ...   
2  Unemployed and actively looking for a job     9      

In [4]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import OLSInfluence
import matplotlib.pyplot as plt

def rreg2_py(df, lhs, rhs,
             toleran=0.01, tune=7.0, iterate=1000,
             genwt=None, no_const=True, graph=False, verbose=True):
    """
    Python reimplementation of Stata rreg2 algorithm (Huber -> biweight).
    Returns dict with final regression results, weights, predicted y, iterations info.

    Parameters
    ----------
    df : pandas.DataFrame
    lhs : str
        Dependent variable name
    rhs : list[str] or str
        Regressor names (if string, whitespace/comma-separated)
    toleran : float
        Convergence tolerance (Stata default .01)
    tune : float
        Tuning constant (Stata used 7 by default; algorithm rescales tune by 4.685/7)
    iterate : int
        Max number of iterations
    genwt : str or None
        If not None, DataFrame column name to store final weights
    no_const : bool
        If True, regressions are done without an intercept (to match Stata code).
    graph : bool
        If True, plot new weight vs old weight during iterations
    verbose : bool
        Print iteration messages
    """
    # Normalize rhs
    if isinstance(rhs, str):
        rhs = [s.strip() for s in rhs.replace(',', ' ').split()]
    X = df[rhs].copy()
    y = df[lhs].copy()

    # Drop rows with missing on X or y (Stata marks them out)
    mask = ~(X.isnull().any(axis=1) | y.isnull())
    X = X.loc[mask].copy()
    y = y.loc[mask].copy()
    nobs = len(y)
    if nobs == 0:
        raise ValueError("No observations after dropping missing.")

    # Optionally add constant (Stata code used noconst; default to match Stata)
    if not no_const:
        X = sm.add_constant(X, has_constant='add')

    # Stata rescales tune: local tune = `tune'*4.685/7
    tune = float(tune) * 4.685 / 7.0

    # 1) initial OLS to get residuals and Cook's D
    ols0 = sm.OLS(y, X).fit()
    infl = OLSInfluence(ols0)
    cooks_d = infl.cooks_distance[0]
    # Omit observations with Cook's D > 1 (Stata)
    keep_mask = cooks_d <= 1
    if keep_mask.sum() < len(keep_mask):
        if verbose:
            print(f"Removed {len(keep_mask) - keep_mask.sum()} observations with Cook's D > 1")

    X = X.loc[keep_mask].copy()
    y = y.loc[keep_mask].copy()
    nobs = len(y)

    # Refit OLS on remaining
    ols1 = sm.OLS(y, X).fit()
    resid = ols1.resid.values  # residuals array
    # median of residuals (r(p50) in Stata)
    rmedian = np.median(resid)
    absdev = np.abs(resid - rmedian)
    # Initialize weights=1
    weight = np.ones_like(resid)
    max_diff = 1.0
    it = 1

    # Huber-like iteration (Stata uses a two-step "2*median" rule first)
    while max_diff > 5 * toleran and it <= iterate:
        oldw = weight.copy()
        # Recompute median-based rule (Stata: if abs(resid)>2*median => 2*median/abs(resid) else 1)
        weight = np.where(np.abs(resid) > 2.0 * rmedian,
                          2.0 * rmedian / np.abs(resid),
                          1.0)
        # Weighted regression (statsmodels.WLS uses weights w_i to minimize sum w_i*(y - Xb)^2)
        wls = sm.WLS(y, X, weights=weight).fit()
        resid = wls.resid.values
        rmedian = np.median(resid)
        absdev = np.abs(resid - rmedian)
        max_diff = np.max(np.abs(weight - oldw))
        if verbose:
            print(f"Huber iteration {it}: max weight change = {max_diff:.6g}")
        if graph:
            plt.figure()
            plt.scatter(oldw, weight, s=10)
            plt.xlabel("Old weight")
            plt.ylabel("New weight")
            plt.title(f"Huber iteration {it}")
            plt.grid(True)
            plt.show()
        it += 1

    if max_diff > 5 * toleran and verbose:
        print("Warning: Huber iterations did not converge within iterate limit.")

    # Biweight iterations (Tukey's bisquare)
    max_diff = 1.0
    notyet = True
    # scale estimate: r(p50)/.6745 (like Stata)
    scale = rmedian / 0.6745 if rmedian != 0 else np.std(resid) or 1.0
    while (max_diff > toleran and it <= iterate) or notyet:
        notyet = False
        oldw = weight.copy()
        # recompute scale each loop from current residuals (Stata picks p50 to compute scale)
        rmedian = np.median(resid)
        scale = rmedian / 0.6745 if rmedian != 0 else (np.std(resid) or 1.0)
        # Tukey bisquare weights: max(1 - (resid/(tune*scale))^2, 0)^2
        u = resid / (tune * scale)
        weight = np.maximum(1.0 - u ** 2, 0.0) ** 2
        # Check for all weights = 0
        nz = np.count_nonzero(weight)
        if nz == 0:
            raise RuntimeError("All weights went to zero in biweight step.")
        # Refit weighted regression
        wls = sm.WLS(y, X, weights=weight).fit()
        resid = wls.resid.values
        absdev = np.abs(resid - np.median(resid))
        max_diff = np.max(np.abs(weight - oldw))
        if verbose:
            print(f"Biweight iteration {it}: max weight change = {max_diff:.6g}, nonzero w = {nz}")
        if graph:
            plt.figure()
            plt.scatter(oldw, weight, s=10)
            plt.xlabel("Old weight")
            plt.ylabel("New weight")
            plt.title(f"Biweight iteration {it}")
            plt.grid(True)
            plt.show()
        it += 1

    if max_diff > toleran and verbose:
        print("Warning: Biweight iterations did not converge within iterate limit.")

    # After convergence compute aa (mean of absdev) and lambda like Stata:
    aa = np.mean(absdev)
    # e(df_m) is model df (number regressors), e(N) is nobs. Stata uses formula:
    # lambda = 1 + ((e(df_m)+1)/e(N)) * (1-aa)/aa
    # We'll use df_m = number of regressors (columns in X)
    df_m = X.shape[1]
    lambda_adj = 1.0 + ((df_m + 1.0) / nobs) * ((1.0 - aa) / aa) if aa != 0 else 1.0

    # Final predicted y from last weighted model
    yhat = wls.predict(X)
    # Adjust y according to Stata: y = yhat + (lambda*scale/aa)*(resid/scale)*weight
    # Algebra simplifies: factor = (lambda/aa)*weight
    if aa == 0:
        factor = 0.0
    else:
        factor = (lambda_adj / aa) * weight
    y_adj = yhat + (resid) * factor  # resid/scale * scale cancels

    # Final regression of y_adj on X (Stata regresses adjusted y on rhs, noconst)
    final_ols = sm.OLS(y_adj, X).fit()
    # Optionally save weights back to DataFrame (aligned with original df indices)
    # Need to create a weights series aligned to original df rows (NaN for dropped rows)
    full_weights = pd.Series(np.nan, index=df.index)
    kept_idx = mask[mask].index[keep_mask.values] if isinstance(mask, pd.Series) else X.index
    # The selection of kept rows got complicated; best approach: map X.index back into df.index
    # X has the original df index (we carried it through). So:
    full_weights.loc[X.index] = weight

    if genwt is not None:
        df[genwt] = full_weights

    out = {
        "final_results": final_ols,
        "final_weights": full_weights,
        "y_adj": pd.Series(y_adj, index=X.index),
        "yhat": pd.Series(yhat, index=X.index),
        "last_wls": wls,
        "iterations": it - 1,
        "lambda": lambda_adj,
        "aa": aa,
        "scale": scale,
        "original_ols": ols1
    }
    return out


In [7]:
import statsmodels.api as sm

def run_rreg(df, varlist, method="huber", maxiter=1000, tol=1e-5):
    """
    Mimics Stata rreg2 variable handling:
    - varlist[0] = dependent variable (y)
    - varlist[1:] = independent variables (X)
    - method = 'huber' or 'biweight'
    """
    # Pick variables
    lhs = varlist[0]         # dependent variable
    rhs = varlist[1:]        # independent variables
    
    y = df[lhs]
    X = df[rhs]
    X = sm.add_constant(X)
    
    # Pick robust estimator
    if method == "huber":
        norm = sm.robust.norms.HuberT()
    elif method == "biweight":
        norm = sm.robust.norms.TukeyBiweight()
    else:
        raise ValueError("method must be 'huber' or 'biweight'")
    
    model = sm.RLM(y, X, M=norm)
    results = model.fit(maxiter=maxiter, tol=tol)
    
    return results


In [8]:
results = run_rreg(df, ["wage", "educ", "exper", "tenure"], method="huber")
print(results.summary())


KeyError: 'wage'

In [9]:
print(df.columns.tolist())

['country', 'pr2010', 'pr2110', 'age', 'b2010', 'b3300', 'b7010', 'q2021', 'wave', 'j1020', 'k1010_1', 'k1010_2', 'k1010_3', 'k2000', 'k2020_1', 'k2020_2', 'k2020_3', 'k2110_1', 'k2110_2', 'k2110_3', 'k2120_1', 'k2120_2', 'k2120_3', 'treatment', 'educ_3', 'hid', 'hhsize', 'male', 'dedu1', 'dedu2', 'dedu3', 'lhhnetinc', 'eincgr', 'liquid', 'buypm_hm', 'buypm_car', 'buypm_dur', 'buypm_holid', 'buypm_lux', 'buypm_oth', 'buypm_none', 'pbuy_hm', 'pbuy_car', 'pbuy_dur', 'pbuy_holid', 'pbuy_lux', 'pbuy_oth', 'pbuy_none', 'eincgr_imean', 'eincgr_istd', 'cons_dbt', 'cons_tot', 'bs_food', 'bs_foodin', 'bs_foodout', 'bs_hous', 'bs_utl', 'bs_hmeqp', 'bs_dbt', 'bs_cloth', 'bs_hlth', 'bs_transp', 'bs_recr', 'bs_edu', 'bs_oth', 'shinv_sav', 'shinv_stock', 'shinv_mutf', 'shinv_ira', 'shinv_stbond', 'shinv_ltbond', 'shinv_crypto', 'ostock', 'omutf', 'sh_sav', 'sh_stock', 'sh_mutf', 'sh_ira', 'sh_bond', 'sh_oth', 'egrea_iskew_bt', 'egrea_iiqr_bt', 'egrea_imean_pt', 'egrea_ivar_pt', 'egrea_istd_pt', 'egr

In [10]:
# Suppose you want to regress total consumption on age, education, treatment, and household net income
results = run_rreg(
    df,
    ["cons_tot", "age", "educ_3", "treatment", "lhhnetinc"],
    method="huber"
)

print(results.summary())


ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).

In [11]:
print(df[["cons_tot", "age", "educ_3", "treatment", "lhhnetinc"]].dtypes)

cons_tot      float64
age           float64
educ_3       category
treatment    category
lhhnetinc     float64
dtype: object


In [12]:
non_numeric_cols = df.select_dtypes(include=['category', 'object']).columns.tolist()
print(non_numeric_cols)

['country', 'pr2010', 'pr2110', 'b2010', 'b3300', 'q2021', 'j1020', 'k2000', 'treatment', 'educ_3', 'liquid', 'buypm_hm', 'buypm_car', 'buypm_dur', 'buypm_holid', 'buypm_lux', 'buypm_oth', 'buypm_none', 'pbuy_hm', 'pbuy_car', 'pbuy_dur', 'pbuy_holid', 'pbuy_lux', 'pbuy_oth', 'pbuy_none']


In [13]:
import pandas as pd

# List of all your variables
vars_to_use = ['country', 'pr2010', 'pr2110', 'b2010', 'b3300', 'q2021', 'j1020', 'k2000',
               'treatment', 'educ_3', 'liquid', 'buypm_hm', 'buypm_car', 'buypm_dur',
               'buypm_holid', 'buypm_lux', 'buypm_oth', 'buypm_none', 'pbuy_hm', 'pbuy_car',
               'pbuy_dur', 'pbuy_holid', 'pbuy_lux', 'pbuy_oth', 'pbuy_none']

# Subset the dataframe
df_sub = df[vars_to_use]

# Automatically find categorical columns
cat_cols = df_sub.select_dtypes(include=['category', 'object']).columns.tolist()

# One-hot encode categorical columns
df_encoded = pd.get_dummies(df_sub, columns=cat_cols, drop_first=True)


In [14]:
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.api as sm

# Load the Stata file
df = pd.read_stata(r"C:\Users\Zach\PycharmProjects\AppliedML\ces_gdp_rct_v1a.dta")

# Create ID and set wave as time variable (similar to tsset)
df['ID'] = df['hid']
df['wave'] = df['wave']  # assuming 'wave' already exists

# Winsorize income
def winsorize_series(s, lower=1, upper=99):
    lower_val = np.percentile(s.dropna(), lower)
    upper_val = np.percentile(s.dropna(), upper)
    return s.clip(lower_val, upper_val)

df['income0_winsorized'] = winsorize_series(df['b7010']) / 1000

# Create nondurable consumption variable
df['nondur'] = df['cons_tot'] - df['cons_dbt']
df['nondur_w'] = winsorize_series(df['nondur'])
# Lagged version (L2)
df['nondur_w_L2'] = df.groupby('ID')['nondur_w'].shift(2) / 1000

# Generate dummies for categorical variables b2010 and b3300
for var in range(1, 7):
    df[f'b2010_v{var}'] = (df['b2010'] == var).astype(float)

for var in range(1, 4):
    df[f'b3300_v{var}'] = (df['b3300'] == var).astype(float)

# List of variables for continuous summary
cont_vars = ['age', 'hhsize', 'income0_winsorized', 'nondur_w_L2']

# Prepare output table
table = []

def weighted_mean_sd(x, weights):
    mean = np.average(x, weights=weights)
    variance = np.average((x - mean)**2, weights=weights)
    return mean, np.sqrt(variance)

# Continuous variables summary
for var in cont_vars:
    means = []
    sds = []
    for jj in range(1, 6):
        subset = df[df['k2000'] == jj]
        w = subset['wgt']
        m, sd = weighted_mean_sd(subset[var], w)
        means.append(m)
        sds.append(sd)
    # Overall mean and sd
    m_all, sd_all = weighted_mean_sd(df[var], df['wgt'])
    means.append(m_all)
    sds.append(sd_all)
    # ANOVA-like test (weighted F test approximation)
    groups = [df[df['k2000'] == jj][var] for jj in range(1, 6)]
    pval = stats.f_oneway(*groups).pvalue
    # Add to table
    table.append([f"{var}: mean"] + means + [pval])
    table.append([f"{var}: std"] + sds + [np.nan])

# Categorical variables summary
cat_vars = ['male'] + [f'b2010_v{i}' for i in range(1,7)] + ['dedu1','dedu2','dedu3'] + \
           [f'b3300_v{i}' for i in range(1,4)] + [f'cnt{i}' for i in range(1,7)]

for var in cat_vars:
    means = []
    sds = []
    for jj in range(1, 6):
        subset = df[df['k2000'] == jj]
        w = subset['wgt']
        m, sd = weighted_mean_sd(subset[var], w)
        means.append(m)
        sds.append(sd)
    # Overall mean and sd
    m_all, sd_all = weighted_mean_sd(df[var], df['wgt'])
    means.append(m_all)
    sds.append(sd_all)
    # ANOVA-like test
    groups = [df[df['k2000'] == jj][var] for jj in range(1, 6)]
    pval = stats.f_oneway(*groups).pvalue
    table.append([var] + means + [pval])

# Number of observations per group
Nobs = []
for jj in range(1, 6):
    Nobs.append(len(df[df['k2000'] == jj]))
Nobs.append(len(df))
table.append(['Nobs'] + Nobs + [np.nan])

# Convert table to DataFrame and save
columns = ['Variable'] + [f'k2000_{i}' for i in range(1,6)] + ['All_obs', 'pval']
table_df = pd.DataFrame(table, columns=columns)
table_df.to_csv(r"C:\Users\Zach\PycharmProjects\AppliedML\Table01.csv", index=False)


C:\Users\Zach\AppData\Local\Temp\ipykernel_84676\1531131472.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'b2010_v{var}'] = (df['b2010'] == var).astype(float)
C:\Users\Zach\AppData\Local\Temp\ipykernel_84676\1531131472.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'b2010_v{var}'] = (df['b2010'] == var).astype(float)
C:\Users\Zach\AppData\Local\Temp\ipykernel_84676\1531131472.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has

ZeroDivisionError: Weights sum to zero, can't be normalized

In [15]:
import pandas as pd

# Assuming your DataFrame is called df
print(df['wgt'])


0                  NaN
1          6893.592285
2          6923.045898
3                  NaN
4          6985.968750
              ...     
102746    15850.798828
102747    14772.033203
102748             NaN
102749    14389.958008
102750    14814.514648
Name: wgt, Length: 102751, dtype: float32


In [16]:
import pandas as pd

# Load the .dta file
file_path = r"C:\Users\Zach\PycharmProjects\AppliedML\ces_gdp_rct_v1a.dta"
df = pd.read_stata(file_path)

# Save as CSV
csv_path = r"C:\Users\Zach\PycharmProjects\AppliedML\ces_gdp_rct_v1a.csv"
df.to_csv(csv_path, index=False)

print(f"CSV saved to {csv_path}")


CSV saved to C:\Users\Zach\PycharmProjects\AppliedML\ces_gdp_rct_v1a.csv


In [17]:
df_collapsed = df.groupby('hid').first().reset_index()


In [18]:
import pandas as pd

# Load the .dta file
file_path = r"C:\Users\Zach\PycharmProjects\AppliedML\ces_gdp_rct_v1a.dta"
df = pd.read_stata(file_path)

# Save as CSV
csv_path = r"C:\Users\Zach\PycharmProjects\AppliedML\ces_gdp_rct_v1a(2).csv"
df.to_csv(csv_path, index=False)

print(f"CSV saved to {csv_path}")


CSV saved to C:\Users\Zach\PycharmProjects\AppliedML\ces_gdp_rct_v1a(2).csv


In [19]:
# Assuming you already did this:
df_collapsed = df.groupby('hid').first().reset_index()

# Save the collapsed DataFrame to a new CSV
df_collapsed.to_csv(r"C:\Users\Zach\PycharmProjects\AppliedML\ces_gdp_rct_v1a_merged.csv", index=False)

In [20]:
import pandas as pd

# Load your CSV
df = pd.read_csv(r"C:\Users\Zach\PycharmProjects\AppliedML\ces_gdp_rct_v1a.csv")

# Identify the columns to pivot (exclude 'hid' and 'wave')
cols_to_pivot = [col for col in df.columns if col not in ['hid', 'wave']]

# Start with an empty DataFrame for the final collapsed data
df_collapsed = pd.DataFrame({'hid': df['hid'].unique()})

# Loop through each variable and pivot it by wave
for var in cols_to_pivot:
    pivoted = df.pivot(index='hid', columns='wave', values=var)
    # Rename columns to include variable name and wave
    pivoted.columns = [f"{var}_wave{int(col)}" for col in pivoted.columns]
    # Merge into the collapsed DataFrame
    df_collapsed = df_collapsed.merge(pivoted, on='hid', how='left')

# Drop columns that are entirely NaN
df_collapsed = df_collapsed.dropna(axis=1, how='all')

# Save the cleaned, collapsed dataset
df_collapsed.to_csv(r"C:\Users\Zach\PycharmProjects\AppliedML\ces_gdp_rct_v1a_hid_collapsed.csv", index=False)

print("Collapsed dataset saved. Shape:", df_collapsed.shape)


Collapsed dataset saved. Shape: (16432, 492)


In [21]:
import pandas as pd

# Load your data
df = pd.read_csv(r"C:\Users\Zach\PycharmProjects\AppliedML\ces_gdp_rct_v1a.csv")

# Identify columns
cols_to_exclude = ['hid', 'wave']

# Separate variables that vary by wave vs. constant variables
wave_varying = []  # variables that should have separate columns per wave
constant_vars = [] # variables that are the same across waves

# Simple heuristic: if a variable has different non-missing values across waves for some households, consider it wave-varying
for col in df.columns:
    if col in cols_to_exclude:
        continue
    if df.groupby('hid')[col].nunique(dropna=True).max() > 1:
        wave_varying.append(col)
    else:
        constant_vars.append(col)

print("Wave-varying variables:", wave_varying)
print("Constant variables:", constant_vars)

# --- Process wave-varying variables ---
df_collapsed = pd.DataFrame({'hid': df['hid'].unique()})

for var in wave_varying:
    pivoted = df.pivot(index='hid', columns='wave', values=var)
    pivoted.columns = [f"{var}_wave{int(col)}" for col in pivoted.columns]
    df_collapsed = df_collapsed.merge(pivoted, on='hid', how='left')

# --- Process constant variables ---
for var in constant_vars:
    first_values = df.groupby('hid')[var].first().reset_index()
    df_collapsed = df_collapsed.merge(first_values, on='hid', how='left')

# Drop columns that are entirely NaN
df_collapsed = df_collapsed.dropna(axis=1, how='all')

# Save the cleaned dataset
df_collapsed.to_csv(r"C:\Users\Zach\PycharmProjects\AppliedML\ces_gdp_rct_v1a_hid_collapsed_clean.csv", index=False)

print("Collapsed dataset saved. Shape:", df_collapsed.shape)


Wave-varying variables: ['q2021', 'k2110_1', 'k2110_2', 'k2110_3', 'k2120_1', 'k2120_2', 'k2120_3', 'treatment', 'eincgr', 'liquid', 'buypm_hm', 'buypm_car', 'buypm_dur', 'buypm_holid', 'buypm_lux', 'buypm_oth', 'buypm_none', 'pbuy_hm', 'pbuy_car', 'pbuy_dur', 'pbuy_holid', 'pbuy_lux', 'pbuy_oth', 'pbuy_none', 'eincgr_imean', 'eincgr_istd', 'cons_dbt', 'cons_tot', 'bs_food', 'bs_foodin', 'bs_foodout', 'bs_hous', 'bs_utl', 'bs_hmeqp', 'bs_dbt', 'bs_cloth', 'bs_hlth', 'bs_transp', 'bs_recr', 'bs_edu', 'bs_oth', 'ostock', 'omutf', 'sh_sav', 'sh_stock', 'sh_mutf', 'sh_ira', 'sh_bond', 'sh_oth', 'eincgr_imean_pt', 'eincgr_ivar_pt', 'eincgr_istd_pt', 'wgt']
Constant variables: ['country', 'pr2010', 'pr2110', 'age', 'b2010', 'b3300', 'b7010', 'j1020', 'k1010_1', 'k1010_2', 'k1010_3', 'k2000', 'k2020_1', 'k2020_2', 'k2020_3', 'educ_3', 'hhsize', 'male', 'dedu1', 'dedu2', 'dedu3', 'lhhnetinc', 'shinv_sav', 'shinv_stock', 'shinv_mutf', 'shinv_ira', 'shinv_stbond', 'shinv_ltbond', 'shinv_crypto',

In [22]:
import pandas as pd

# Load the dataset
df = pd.read_csv(r"C:\Users\Zach\PycharmProjects\AppliedML\ces_gdp_rct_v1a.csv")

# Exclude identifiers
cols_to_exclude = ['hid', 'wave']

# Separate wave-varying vs constant variables
wave_varying = []
constant_vars = []

for col in df.columns:
    if col in cols_to_exclude:
        continue
    if df.groupby('hid')[col].nunique(dropna=True).max() > 1:
        wave_varying.append(col)
    else:
        constant_vars.append(col)

# Initialize collapsed DataFrame
df_collapsed = pd.DataFrame({'hid': df['hid'].unique()})

# --- 1. Process wave-varying variables ---
for var in wave_varying:
    pivoted = df.pivot(index='hid', columns='wave', values=var)
    pivoted.columns = [f"{var}_wave{int(col)}" for col in pivoted.columns]
    
    # Forward-fill and backward-fill per household
    pivoted = pivoted.ffill(axis=1).bfill(axis=1)
    
    df_collapsed = df_collapsed.merge(pivoted, on='hid', how='left')

# --- 2. Process constant variables ---
for var in constant_vars:
    first_values = df.groupby('hid')[var].first().reset_index()
    df_collapsed = df_collapsed.merge(first_values, on='hid', how='left')

# --- 3. Convert Yes/No to 1/0 ---
df_collapsed = df_collapsed.replace({'Yes': 1, 'No': 0})

# --- 4. Convert non-numeric, non-Yes/No categorical variables into dummies ---
for col in df_collapsed.columns:
    if col == 'hid':
        continue
    if df_collapsed[col].dtype == 'object':
        # Create dummy variables
        dummies = pd.get_dummies(df_collapsed[col], prefix=col)
        df_collapsed = pd.concat([df_collapsed.drop(columns=[col]), dummies], axis=1)

# --- 5. Drop columns that are completely NaN (optional) ---
df_collapsed = df_collapsed.dropna(axis=1, how='all')

# Save the cleaned dataset
df_collapsed.to_csv(r"C:\Users\Zach\PycharmProjects\AppliedML\ces_gdp_rct_v1a_hid_clean.csv", index=False)

print("Cleaning complete! Dataset shape:", df_collapsed.shape)


Cleaning complete! Dataset shape: (16432, 756)


In [23]:
import pandas as pd

# Load dataset
df = pd.read_csv(r"C:\Users\Zach\PycharmProjects\AppliedML\ces_gdp_rct_v1a.csv")

# Exclude identifiers
cols_to_exclude = ['hid', 'wave']

# Separate wave-varying vs constant variables
wave_varying = []
constant_vars = []

for col in df.columns:
    if col in cols_to_exclude:
        continue
    if df.groupby('hid')[col].nunique(dropna=True).max() > 1:
        wave_varying.append(col)
    else:
        constant_vars.append(col)

# Initialize collapsed DataFrame
df_collapsed = pd.DataFrame({'hid': df['hid'].unique()})

# --- 1. Process wave-varying variables ---
for var in wave_varying:
    pivoted = df.pivot(index='hid', columns='wave', values=var)
    # Forward-fill and backward-fill per household
    pivoted = pivoted.ffill(axis=1).bfill(axis=1)
    # Rename columns to include wave
    pivoted.columns = [f"{var}_wave{int(col)}" for col in pivoted.columns]
    
    # Convert Yes/No to 1/0
    pivoted = pivoted.replace({'Yes': 1, 'No': 0})
    
    # Convert non-numeric columns to dummies
    for col in pivoted.columns:
        if pivoted[col].dtype == 'object':
            dummies = pd.get_dummies(pivoted[col], prefix=col)
            pivoted = pivoted.drop(columns=[col])
            pivoted = pd.concat([pivoted, dummies], axis=1)
    
    df_collapsed = df_collapsed.merge(pivoted, on='hid', how='left')

# --- 2. Process constant variables ---
for var in constant_vars:
    first_values = df.groupby('hid')[var].first().reset_index()
    # Convert Yes/No to 1/0
    first_values[var] = first_values[var].replace({'Yes': 1, 'No': 0})
    # Convert non-numeric columns to dummies
    if first_values[var].dtype == 'object':
        dummies = pd.get_dummies(first_values[var], prefix=var)
        first_values = first_values.drop(columns=[var])
        first_values = pd.concat([first_values, dummies], axis=1)
    df_collapsed = df_collapsed.merge(first_values, on='hid', how='left')

# --- 3. Drop rows with too many missing values (e.g., >50% NaN) ---
threshold = df_collapsed.shape[1] * 0.5
df_collapsed = df_collapsed.dropna(thresh=threshold)

# --- 4. Drop columns that are completely NaN ---
df_collapsed = df_collapsed.dropna(axis=1, how='all')

# --- 5. Save cleaned dataset ---
df_collapsed.to_csv(r"C:\Users\Zach\PycharmProjects\AppliedML\ces_gdp_rct_v1a_hid_clean_final.csv", index=False)

print("Cleaning complete! Shape:", df_collapsed.shape)


Cleaning complete! Shape: (15540, 756)


In [24]:
import pandas as pd

# Load dataset
df = pd.read_csv(r"C:\Users\Zach\PycharmProjects\AppliedML\ces_gdp_rct_v1a.csv")

# Exclude identifiers
cols_to_exclude = ['hid', 'wave']

# Separate wave-varying vs constant variables
wave_varying = []
constant_vars = []

for col in df.columns:
    if col in cols_to_exclude:
        continue
    if df.groupby('hid')[col].nunique(dropna=True).max() > 1:
        wave_varying.append(col)
    else:
        constant_vars.append(col)

# Initialize collapsed DataFrame
df_collapsed = pd.DataFrame({'hid': df['hid'].unique()})

# --- 1. Process wave-varying variables ---
for var in wave_varying:
    pivoted = df.pivot(index='hid', columns='wave', values=var)
    pivoted = pivoted.ffill(axis=1).bfill(axis=1)
    pivoted.columns = [f"{var}_wave{int(col)}" for col in pivoted.columns]
    
    # Convert Yes/No and True/False to 1/0
    pivoted = pivoted.replace({'Yes': 1, 'No': 0, True: 1, False: 0})
    
    # Convert non-numeric columns to dummies
    for col in pivoted.columns:
        if pivoted[col].dtype == 'object':
            dummies = pd.get_dummies(pivoted[col], prefix=col)
            pivoted = pivoted.drop(columns=[col])
            pivoted = pd.concat([pivoted, dummies], axis=1)
    
    df_collapsed = df_collapsed.merge(pivoted, on='hid', how='left')

# --- 2. Process constant variables ---
for var in constant_vars:
    first_values = df.groupby('hid')[var].first().reset_index()
    first_values[var] = first_values[var].replace({'Yes': 1, 'No': 0, True: 1, False: 0})
    
    if first_values[var].dtype == 'object':
        dummies = pd.get_dummies(first_values[var], prefix=var)
        first_values = first_values.drop(columns=[var])
        first_values = pd.concat([first_values, dummies], axis=1)
    
    df_collapsed = df_collapsed.merge(first_values, on='hid', how='left')

# --- 3. Drop rows with too many missing values (>50% NaN) ---
threshold = df_collapsed.shape[1] * 0.5
df_collapsed = df_collapsed.dropna(thresh=threshold)

# --- 4. Drop columns that are completely NaN ---
df_collapsed = df_collapsed.dropna(axis=1, how='all')

# --- 5. Save cleaned dataset ---
df_collapsed.to_csv(r"C:\Users\Zach\PycharmProjects\AppliedML\ces_gdp_rct_v1a_hid_clean_final.csv", index=False)

print("Cleaning complete! Shape:", df_collapsed.shape)


Cleaning complete! Shape: (15540, 756)


In [1]:
import pandas as pd

# Load dataset
df = pd.read_csv(r"C:\Users\Zach\PycharmProjects\AppliedML\ces_gdp_rct_v1a.csv")

# Exclude identifiers
cols_to_exclude = ['hid', 'wave']

# Separate wave-varying vs constant variables
wave_varying = []
constant_vars = []

for col in df.columns:
    if col in cols_to_exclude:
        continue
    if df.groupby('hid')[col].nunique(dropna=True).max() > 1:
        wave_varying.append(col)
    else:
        constant_vars.append(col)

# Initialize collapsed DataFrame
df_collapsed = pd.DataFrame({'hid': df['hid'].unique()})

# --- 1. Process wave-varying variables ---
for var in wave_varying:
    pivoted = df.pivot(index='hid', columns='wave', values=var)
    pivoted = pivoted.ffill(axis=1).bfill(axis=1)
    pivoted.columns = [f"{var}_wave{int(col)}" for col in pivoted.columns]
    
    # Convert Yes/No and True/False to 1/0
    pivoted = pivoted.replace({'Yes': 1, 'No': 0, True: 1, False: 0})
    
    # Convert all columns to numeric (forces True/False to 1/0)
    pivoted = pivoted.apply(pd.to_numeric, errors='ignore')
    
    # Convert non-numeric categorical columns to dummies
    for col in pivoted.columns:
        if pivoted[col].dtype == 'object':
            dummies = pd.get_dummies(pivoted[col], prefix=col)
            pivoted = pivoted.drop(columns=[col])
            pivoted = pd.concat([pivoted, dummies], axis=1)
    
    df_collapsed = df_collapsed.merge(pivoted, on='hid', how='left')

# --- 2. Process constant variables ---
for var in constant_vars:
    first_values = df.groupby('hid')[var].first().reset_index()
    first_values[var] = first_values[var].replace({'Yes': 1, 'No': 0, True: 1, False: 0})
    first_values[var] = pd.to_numeric(first_values[var], errors='ignore')
    
    if first_values[var].dtype == 'object':
        dummies = pd.get_dummies(first_values[var], prefix=var)
        first_values = first_values.drop(columns=[var])
        first_values = pd.concat([first_values, dummies], axis=1)
    
    df_collapsed = df_collapsed.merge(first_values, on='hid', how='left')

# --- 3. Drop all rows with any missing values ---
df_collapsed = df_collapsed.dropna()

# --- 4. Drop columns that are completely NaN (just in case) ---
df_collapsed = df_collapsed.dropna(axis=1, how='all')

# --- 5. Save cleaned dataset ---
df_collapsed.to_csv(r"C:\Users\Zach\PycharmProjects\AppliedML\ces_gdp_rct_v1a_hid_clean_final_no_missing.csv", index=False)

print("Cleaning complete! Shape:", df_collapsed.shape)


Cleaning complete! Shape: (3309, 756)


In [26]:
import pandas as pd

# Load the Excel file
file_path = r"C:\Users\Zach\PycharmProjects\AppliedML\CES Data.xlsx"

# Read only the first two rows
df_names = pd.read_excel(file_path, header=None, nrows=2)

# Extract old and descriptive names
old_names = df_names.iloc[0]
descriptive_names = df_names.iloc[1]

# Filter out pairs where either value is missing
valid_pairs = [(old, new) for old, new in zip(old_names, descriptive_names)
               if pd.notna(old) and pd.notna(new)]

# Print the valid pairs
for old, new in valid_pairs:
    print(f"{old} -> {new}")


country -> Country
pr2010 -> Recruitment Type
pr2110 -> Method & Phe0type
age -> Age
b2010 -> Employment Situation
b3300 -> Housing Type
b7010 -> Total Combined Net of Tax Income Option
q2021 -> Employment Status
wave -> Questionare Wave
j1020 -> Industry 
k1010_1 -> Min Expectations of Euro Area Growth 
k1010_2 -> Max Expectations of Euro Area Growth 
k1010_3 -> Growth uncertainty probability
k2000 -> Treatment Group
k2020_1 -> Probability of low growth rate
k2020_2 -> Probability of medium growth rate
k2020_3 -> Probability of max growth rate
k2110_1 -> Min Household Income Growth
k2110_2 -> Med Household Income Growth
k2110_3 -> Max Household Income Growth
k2120_1 -> Probabilty Min Income Growth
k2120_2 -> Probabilty Med Income Growth
k2120_3 -> Probabilty Max Income Growth
treatment -> Experimental Group
educ_3 -> Education Categories
hid -> Respondent ID
hhsize -> Household Size
male -> Male
dedu1 -> Primary School (most education)
dedu2 -> Secondary School
dedu3 -> Post-Secondary

In [27]:
import pandas as pd

# Load the cleaned dataset
df_clean = pd.read_csv(r"C:\Users\Zach\PycharmProjects\AppliedML\ces_gdp_rct_v1a_hid_clean_final_no_missing.csv")

# Mapping of old variable names -> descriptive names
name_mapping = {
    "country": "Country",
    "pr2010": "Recruitment Type",
    "pr2110": "Method & Phe0type",
    "age": "Age",
    "b2010": "Employment Situation",
    "b3300": "Housing Type",
    "b7010": "Total Combined Net of Tax Income Option",
    "q2021": "Employment Status",
    "wave": "Questionare Wave",
    "j1020": "Industry",
    "k1010_1": "Min Expectations of Euro Area Growth",
    "k1010_2": "Max Expectations of Euro Area Growth",
    "k1010_3": "Growth uncertainty probability",
    "k2000": "Treatment Group",
    "k2020_1": "Probability of low growth rate",
    "k2020_2": "Probability of medium growth rate",
    "k2020_3": "Probability of max growth rate",
    "k2110_1": "Min Household Income Growth",
    "k2110_2": "Med Household Income Growth",
    "k2110_3": "Max Household Income Growth",
    "k2120_1": "Probabilty Min Income Growth",
    "k2120_2": "Probabilty Med Income Growth",
    "k2120_3": "Probabilty Max Income Growth",
    "treatment": "Experimental Group",
    "educ_3": "Education Categories",
    "hid": "Respondent ID",
    "hhsize": "Household Size",
    "male": "Male",
    "dedu1": "Primary School (most education)",
    "dedu2": "Secondary School",
    "dedu3": "Post-Secondary School",
    "eincgr": "Expected Net Household Income Growth in next year",
    "liquid": "Sufficient Liquidity",
    "buypm_hm": "Purchased house/apartment in past",
    "buypm_car": "Purchased car/other vehicle in past",
    "buypm_dur": "Purchased home appliance/furinture in past",
    "buypm_holid": "Purchased home holiday in past",
    "buypm_lux": "Purchased luxury good in past",
    "buypm_oth": "Purchased other major item in past",
    "buypm_0ne": "Purchased 0 major item in past",
    "pbuy_hm": "Plan to buy a home in next year",
    "pbuy_car": "Plan to buy a car in next year",
    "pbuy_dur": "Plan to buy home appliance, furniture, etc in next year",
    "pbuy_holid": "Plan to buy a holiday in next year",
    "pbuy_lux": "Plan to buy a luxury good in next year",
    "pbuy_oth": "Plan to buy other major purchase in next year",
    "pbuy_0ne": "Plan to buy a 0thing major in next year",
    "eincgr_imean": "Mean expected income growth",
    "eincgr_istd": "Standard deviation of expected income growth",
    "cons_dbt": "Debt repayments",
    "cons_tot": "Total expenditure final",
    "bs_food": "Food budget share",
    "bs_foodin": "Inside food budget share",
    "bs_foodout": "Outside food budget share",
    "bs_hous": "House budget share",
    "bs_utl": "Utilities budget share",
    "bs_hmeqp": "Home appliances budget share",
    "bs_dbt": "Debt budget share",
    "bs_cloth": "Clothes budget share",
    "bs_hlth": "Health budget share",
    "bs_transp": "Transportation budget share",
    "bs_recr": "Recreation budget share",
    "bs_edu": "Education budget share",
    "bs_oth": "Other budget share",
    "shinv_sav": "Savings asset share",
    "shinv_stock": "Stocks asset share",
    "shinv_mutf": "Mutual funds asset share",
    "shinv_ira": "Pension asset share",
    "shinv_stbond": "Short-term bonds asset share",
    "shinv_ltbond": "Long-term bonds asset share",
    "shinv_crypto": "Crypto asset share",
    "ostock": "Owns stocks",
    "omutf": "Owns mutual fund",
    "sh_sav": "Savings wealth share",
    "sh_stock": "Stock wealth share",
    "sh_mutf": "Mutual fund wealth share",
    "sh_ira": "Pension wealth share",
    "sh_bond": "Bonds wealth share",
    "sh_oth": "Other wealth share",
    "egrea_iskew_bt": "Skewness of growth",
    "egrea_iiqr_bt": "IQR",
    "egrea_imean_pt": "Mean growth post",
    "egrea_ivar_pt": "Variance growth post",
    "egrea_istd_pt": "Std growth post",
    "egrea_iskew_pt": "Skew post",
    "egrea_iiqr_pt": "IQR post",
    "eincgr_imean_pt": "Mean HH growth most",
    "eincgr_ivar_pt": "Var HH growth post",
    "eincgr_istd_pt": "Std. HH growth post",
    "cnt1": "Belgium",
    "cnt2": "Denmark",
    "cnt3": "Spain",
    "cnt4": "France",
    "cnt5": "Italy",
    "cnt6": "Netherlands",
    "treat1": "Control group",
    "treat2": "Mean treatment",
    "treat3": "Uncertainty Treatment (EU)",
    "treat4": "Both Treatment",
    "treat5": "Country Uncertainty Treatment",
    "egrea_imean_bt": "Mean growth pre",
    "egrea_ivar_bt": "Variance growth pre",
    "egrea_istd_bt": "Std. growth pre",
    "egrea_imeansp_bt": "Mean growth pre (split triangle)",
    "egrea_ivarsp_bt": "Var growth pre (split tri)",
    "egrea_istdsp_bt": "Std. growth pre (split tri)",
    "eincgr_imean_bt": "Mean HH income growth pre",
    "eincgr_ivar_bt": "Var HH income growth pre",
    "eincgr_istd_bt": "STd. HH income growth pre",
    "wgt": "weight"
}

# Rename columns in the cleaned dataset
df_clean = df_clean.rename(columns=name_mapping)

# Save the dataset with descriptive variable names
df_clean.to_csv(r"C:\Users\Zach\PycharmProjects\AppliedML\ces_gdp_rct_v1a_hid_clean_final_no_missing_named.csv", index=False)

print("Variable names replaced successfully. Shape:", df_clean.shape)


Variable names replaced successfully. Shape: (3309, 756)


In [28]:
import pandas as pd

# Load the cleaned dataset with wave-specific columns
df_clean = pd.read_csv(r"C:\Users\Zach\PycharmProjects\AppliedML\ces_gdp_rct_v1a_hid_clean_final_no_missing.csv")

# Mapping of base variable names -> descriptive names
name_mapping = {
    "country": "Country",
    "pr2010": "Recruitment Type",
    "pr2110": "Method & Phe0type",
    "age": "Age",
    "b2010": "Employment Situation",
    "b3300": "Housing Type",
    "b7010": "Total Combined Net of Tax Income Option",
    "q2021": "Employment Status",
    "wave": "Questionare Wave",
    "j1020": "Industry",
    "k1010_1": "Min Expectations of Euro Area Growth",
    "k1010_2": "Max Expectations of Euro Area Growth",
    "k1010_3": "Growth uncertainty probability",
    "k2000": "Treatment Group",
    "k2020_1": "Probability of low growth rate",
    "k2020_2": "Probability of medium growth rate",
    "k2020_3": "Probability of max growth rate",
    "k2110_1": "Min Household Income Growth",
    "k2110_2": "Med Household Income Growth",
    "k2110_3": "Max Household Income Growth",
    "k2120_1": "Probabilty Min Income Growth",
    "k2120_2": "Probabilty Med Income Growth",
    "k2120_3": "Probabilty Max Income Growth",
    "treatment": "Experimental Group",
    "educ_3": "Education Categories",
    "hid": "Respondent ID",
    "hhsize": "Household Size",
    "male": "Male",
    "dedu1": "Primary School (most education)",
    "dedu2": "Secondary School",
    "dedu3": "Post-Secondary School",
    "eincgr": "Expected Net Household Income Growth in next year",
    "liquid": "Sufficient Liquidity",
    "buypm_hm": "Purchased house/apartment in past",
    "buypm_car": "Purchased car/other vehicle in past",
    "buypm_dur": "Purchased home appliance/furinture in past",
    "buypm_holid": "Purchased home holiday in past",
    "buypm_lux": "Purchased luxury good in past",
    "buypm_oth": "Purchased other major item in past",
    "buypm_0ne": "Purchased 0 major item in past",
    "pbuy_hm": "Plan to buy a home in next year",
    "pbuy_car": "Plan to buy a car in next year",
    "pbuy_dur": "Plan to buy home appliance, furniture, etc in next year",
    "pbuy_holid": "Plan to buy a holiday in next year",
    "pbuy_lux": "Plan to buy a luxury good in next year",
    "pbuy_oth": "Plan to buy other major purchase in next year",
    "pbuy_0ne": "Plan to buy a 0thing major in next year",
    "eincgr_imean": "Mean expected income growth",
    "eincgr_istd": "Standard deviation of expected income growth",
    "cons_dbt": "Debt repayments",
    "cons_tot": "Total expenditure final",
    "bs_food": "Food budget share",
    "bs_foodin": "Inside food budget share",
    "bs_foodout": "Outside food budget share",
    "bs_hous": "House budget share",
    "bs_utl": "Utilities budget share",
    "bs_hmeqp": "Home appliances budget share",
    "bs_dbt": "Debt budget share",
    "bs_cloth": "Clothes budget share",
    "bs_hlth": "Health budget share",
    "bs_transp": "Transportation budget share",
    "bs_recr": "Recreation budget share",
    "bs_edu": "Education budget share",
    "bs_oth": "Other budget share",
    "shinv_sav": "Savings asset share",
    "shinv_stock": "Stocks asset share",
    "shinv_mutf": "Mutual funds asset share",
    "shinv_ira": "Pension asset share",
    "shinv_stbond": "Short-term bonds asset share",
    "shinv_ltbond": "Long-term bonds asset share",
    "shinv_crypto": "Crypto asset share",
    "ostock": "Owns stocks",
    "omutf": "Owns mutual fund",
    "sh_sav": "Savings wealth share",
    "sh_stock": "Stock wealth share",
    "sh_mutf": "Mutual fund wealth share",
    "sh_ira": "Pension wealth share",
    "sh_bond": "Bonds wealth share",
    "sh_oth": "Other wealth share",
    "egrea_iskew_bt": "Skewness of growth",
    "egrea_iiqr_bt": "IQR",
    "egrea_imean_pt": "Mean growth post",
    "egrea_ivar_pt": "Variance growth post",
    "egrea_istd_pt": "Std growth post",
    "egrea_iskew_pt": "Skew post",
    "egrea_iiqr_pt": "IQR post",
    "eincgr_imean_pt": "Mean HH growth most",
    "eincgr_ivar_pt": "Var HH growth post",
    "eincgr_istd_pt": "Std. HH growth post",
    "cnt1": "Belgium",
    "cnt2": "Denmark",
    "cnt3": "Spain",
    "cnt4": "France",
    "cnt5": "Italy",
    "cnt6": "Netherlands",
    "treat1": "Control group",
    "treat2": "Mean treatment",
    "treat3": "Uncertainty Treatment (EU)",
    "treat4": "Both Treatment",
    "treat5": "Country Uncertainty Treatment",
    "egrea_imean_bt": "Mean growth pre",
    "egrea_ivar_bt": "Variance growth pre",
    "egrea_istd_bt": "Std. growth pre",
    "egrea_imeansp_bt": "Mean growth pre (split triangle)",
    "egrea_ivarsp_bt": "Var growth pre (split tri)",
    "egrea_istdsp_bt": "Std. growth pre (split tri)",
    "eincgr_imean_bt": "Mean HH income growth pre",
    "eincgr_ivar_bt": "Var HH income growth pre",
    "eincgr_istd_bt": "STd. HH income growth pre",
    "wgt": "weight"
}

# Function to rename columns dynamically
def rename_column(col):
    # Check if the column has the wave/category format
    for old_var in name_mapping.keys():
        if col.startswith(old_var + "_wave"):
            rest = col[len(old_var)+1:]  # e.g., wave8_Working full-time...
            return f"{name_mapping[old_var]}-{rest.replace('_','-')}"
        elif col == old_var:
            return name_mapping[old_var]
    return col  # keep original if no match

# Apply renaming
df_clean = df_clean.rename(columns=lambda x: rename_column(x))

# Save the updated dataset
df_clean.to_csv(r"C:\Users\Zach\PycharmProjects\AppliedML\ces_gdp_rct_v1a_hid_clean_final_named_wave.csv", index=False)

print("Wave-specific categorical columns renamed successfully!")


Wave-specific categorical columns renamed successfully!


In [29]:
import pandas as pd

# Load the cleaned dataset
df_clean = pd.read_csv(r"C:\Users\Zach\PycharmProjects\AppliedML\ces_gdp_rct_v1a_hid_clean_final_no_missing.csv")

# Mapping of base variable names -> descriptive names
name_mapping = {
    "country": "Country",
    "pr2010": "Recruitment Type",
    "pr2110": "Method & Phe0type",
    "age": "Age",
    "b2010": "Employment Situation",
    "b3300": "Housing Type",
    "b7010": "Total Combined Net of Tax Income Option",
    "q2021": "Employment Status",
    "wave": "Questionare Wave",
    "j1020": "Industry",
    "k1010_1": "Min Expectations of Euro Area Growth",
    "k1010_2": "Max Expectations of Euro Area Growth",
    "k1010_3": "Growth uncertainty probability",
    "k2000": "Treatment Group",
    "k2020_1": "Probability of low growth rate",
    "k2020_2": "Probability of medium growth rate",
    "k2020_3": "Probability of max growth rate",
    "k2110_1": "Min Household Income Growth",
    "k2110_2": "Med Household Income Growth",
    "k2110_3": "Max Household Income Growth",
    "k2120_1": "Probabilty Min Income Growth",
    "k2120_2": "Probabilty Med Income Growth",
    "k2120_3": "Probabilty Max Income Growth",
    "treatment": "Experimental Group",
    "educ_3": "Education Categories",
    "hid": "Respondent ID",
    "hhsize": "Household Size",
    "male": "Male",
    "dedu1": "Primary School (most education)",
    "dedu2": "Secondary School",
    "dedu3": "Post-Secondary School",
    "eincgr": "Expected Net Household Income Growth in next year",
    "liquid": "Sufficient Liquidity",
    "buypm_hm": "Purchased house/apartment in past",
    "buypm_car": "Purchased car/other vehicle in past",
    "buypm_dur": "Purchased home appliance/furinture in past",
    "buypm_holid": "Purchased home holiday in past",
    "buypm_lux": "Purchased luxury good in past",
    "buypm_oth": "Purchased other major item in past",
    "buypm_0ne": "Purchased 0 major item in past",
    "pbuy_hm": "Plan to buy a home in next year",
    "pbuy_car": "Plan to buy a car in next year",
    "pbuy_dur": "Plan to buy home appliance, furniture, etc in next year",
    "pbuy_holid": "Plan to buy a holiday in next year",
    "pbuy_lux": "Plan to buy a luxury good in next year",
    "pbuy_oth": "Plan to buy other major purchase in next year",
    "pbuy_0ne": "Plan to buy a 0thing major in next year",
    "eincgr_imean": "Mean expected income growth",
    "eincgr_istd": "Standard deviation of expected income growth",
    "cons_dbt": "Debt repayments",
    "cons_tot": "Total expenditure final",
    "bs_food": "Food budget share",
    "bs_foodin": "Inside food budget share",
    "bs_foodout": "Outside food budget share",
    "bs_hous": "House budget share",
    "bs_utl": "Utilities budget share",
    "bs_hmeqp": "Home appliances budget share",
    "bs_dbt": "Debt budget share",
    "bs_cloth": "Clothes budget share",
    "bs_hlth": "Health budget share",
    "bs_transp": "Transportation budget share",
    "bs_recr": "Recreation budget share",
    "bs_edu": "Education budget share",
    "bs_oth": "Other budget share",
    "shinv_sav": "Savings asset share",
    "shinv_stock": "Stocks asset share",
    "shinv_mutf": "Mutual funds asset share",
    "shinv_ira": "Pension asset share",
    "shinv_stbond": "Short-term bonds asset share",
    "shinv_ltbond": "Long-term bonds asset share",
    "shinv_crypto": "Crypto asset share",
    "ostock": "Owns stocks",
    "omutf": "Owns mutual fund",
    "sh_sav": "Savings wealth share",
    "sh_stock": "Stock wealth share",
    "sh_mutf": "Mutual fund wealth share",
    "sh_ira": "Pension wealth share",
    "sh_bond": "Bonds wealth share",
    "sh_oth": "Other wealth share",
    "egrea_iskew_bt": "Skewness of growth",
    "egrea_iiqr_bt": "IQR",
    "egrea_imean_pt": "Mean growth post",
    "egrea_ivar_pt": "Variance growth post",
    "egrea_istd_pt": "Std growth post",
    "egrea_iskew_pt": "Skew post",
    "egrea_iiqr_pt": "IQR post",
    "eincgr_imean_pt": "Mean HH growth most",
    "eincgr_ivar_pt": "Var HH growth post",
    "eincgr_istd_pt": "Std. HH growth post",
    "cnt1": "Belgium",
    "cnt2": "Denmark",
    "cnt3": "Spain",
    "cnt4": "France",
    "cnt5": "Italy",
    "cnt6": "Netherlands",
    "treat1": "Control group",
    "treat2": "Mean treatment",
    "treat3": "Uncertainty Treatment (EU)",
    "treat4": "Both Treatment",
    "treat5": "Country Uncertainty Treatment",
    "egrea_imean_bt": "Mean growth pre",
    "egrea_ivar_bt": "Variance growth pre",
    "egrea_istd_bt": "Std. growth pre",
    "egrea_imeansp_bt": "Mean growth pre (split triangle)",
    "egrea_ivarsp_bt": "Var growth pre (split tri)",
    "egrea_istdsp_bt": "Std. growth pre (split tri)",
    "eincgr_imean_bt": "Mean HH income growth pre",
    "eincgr_ivar_bt": "Var HH income growth pre",
    "eincgr_istd_bt": "STd. HH income growth pre",
    "wgt": "weight"
}

# Create a second row with descriptive names
descriptive_row = [name_mapping.get(col, "") for col in df_clean.columns]

# Create a new DataFrame where first row = original columns, second row = descriptive names
df_with_header = pd.DataFrame([df_clean.columns.tolist(), descriptive_row] + df_clean.values.tolist())

# Save to CSV (no automatic header, because first row already contains it)
df_with_header.to_csv(r"C:\Users\Zach\PycharmProjects\AppliedML\ces_gdp_rct_v1a_with_descriptive_header.csv",
                      index=False, header=False)

print("CSV saved with descriptive variable names in second row.")


CSV saved with descriptive variable names in second row.


In [31]:
import pandas as pd

# Load the cleaned dataset
df_clean = pd.read_csv(r"C:\Users\Zach\PycharmProjects\AppliedML\ces_gdp_rct_v1a_hid_clean_final_no_missing.csv")

# Mapping of base variable names -> descriptive names
name_mapping = {
    "country": "Country",
    "pr2010": "Recruitment Type",
    "pr2110": "Method & Phe0type",
    "age": "Age",
    "b2010": "Employment Situation",
    "b3300": "Housing Type",
    "b7010": "Total Combined Net of Tax Income Option",
    "q2021": "Employment Status",
    "wave": "Questionare Wave",
    "j1020": "Industry",
    "k1010_1": "Min Expectations of Euro Area Growth",
    "k1010_2": "Max Expectations of Euro Area Growth",
    "k1010_3": "Growth uncertainty probability",
    "k2000": "Treatment Group",
    "k2020_1": "Probability of low growth rate",
    "k2020_2": "Probability of medium growth rate",
    "k2020_3": "Probability of max growth rate",
    "k2110_1": "Min Household Income Growth",
    "k2110_2": "Med Household Income Growth",
    "k2110_3": "Max Household Income Growth",
    "k2120_1": "Probabilty Min Income Growth",
    "k2120_2": "Probabilty Med Income Growth",
    "k2120_3": "Probabilty Max Income Growth",
    "treatment": "Experimental Group",
    "educ_3": "Education Categories",
    "hid": "Respondent ID",
    "hhsize": "Household Size",
    "male": "Male",
    "dedu1": "Primary School (most education)",
    "dedu2": "Secondary School",
    "dedu3": "Post-Secondary School",
    "eincgr": "Expected Net Household Income Growth in next year",
    "liquid": "Sufficient Liquidity",
    "buypm_hm": "Purchased house/apartment in past",
    "buypm_car": "Purchased car/other vehicle in past",
    "buypm_dur": "Purchased home appliance/furinture in past",
    "buypm_holid": "Purchased home holiday in past",
    "buypm_lux": "Purchased luxury good in past",
    "buypm_oth": "Purchased other major item in past",
    "buypm_0ne": "Purchased 0 major item in past",
    "pbuy_hm": "Plan to buy a home in next year",
    "pbuy_car": "Plan to buy a car in next year",
    "pbuy_dur": "Plan to buy home appliance, furniture, etc in next year",
    "pbuy_holid": "Plan to buy a holiday in next year",
    "pbuy_lux": "Plan to buy a luxury good in next year",
    "pbuy_oth": "Plan to buy other major purchase in next year",
    "pbuy_0ne": "Plan to buy a 0thing major in next year",
    "eincgr_imean": "Mean expected income growth",
    "eincgr_istd": "Standard deviation of expected income growth",
    "cons_dbt": "Debt repayments",
    "cons_tot": "Total expenditure final",
    "bs_food": "Food budget share",
    "bs_foodin": "Inside food budget share",
    "bs_foodout": "Outside food budget share",
    "bs_hous": "House budget share",
    "bs_utl": "Utilities budget share",
    "bs_hmeqp": "Home appliances budget share",
    "bs_dbt": "Debt budget share",
    "bs_cloth": "Clothes budget share",
    "bs_hlth": "Health budget share",
    "bs_transp": "Transportation budget share",
    "bs_recr": "Recreation budget share",
    "bs_edu": "Education budget share",
    "bs_oth": "Other budget share",
    "shinv_sav": "Savings asset share",
    "shinv_stock": "Stocks asset share",
    "shinv_mutf": "Mutual funds asset share",
    "shinv_ira": "Pension asset share",
    "shinv_stbond": "Short-term bonds asset share",
    "shinv_ltbond": "Long-term bonds asset share",
    "shinv_crypto": "Crypto asset share",
    "ostock": "Owns stocks",
    "omutf": "Owns mutual fund",
    "sh_sav": "Savings wealth share",
    "sh_stock": "Stock wealth share",
    "sh_mutf": "Mutual fund wealth share",
    "sh_ira": "Pension wealth share",
    "sh_bond": "Bonds wealth share",
    "sh_oth": "Other wealth share",
    "egrea_iskew_bt": "Skewness of growth",
    "egrea_iiqr_bt": "IQR",
    "egrea_imean_pt": "Mean growth post",
    "egrea_ivar_pt": "Variance growth post",
    "egrea_istd_pt": "Std growth post",
    "egrea_iskew_pt": "Skew post",
    "egrea_iiqr_pt": "IQR post",
    "eincgr_imean_pt": "Mean HH growth most",
    "eincgr_ivar_pt": "Var HH growth post",
    "eincgr_istd_pt": "Std. HH growth post",
    "cnt1": "Belgium",
    "cnt2": "Denmark",
    "cnt3": "Spain",
    "cnt4": "France",
    "cnt5": "Italy",
    "cnt6": "Netherlands",
    "treat1": "Control group",
    "treat2": "Mean treatment",
    "treat3": "Uncertainty Treatment (EU)",
    "treat4": "Both Treatment",
    "treat5": "Country Uncertainty Treatment",
    "egrea_imean_bt": "Mean growth pre",
    "egrea_ivar_bt": "Variance growth pre",
    "egrea_istd_bt": "Std. growth pre",
    "egrea_imeansp_bt": "Mean growth pre (split triangle)",
    "egrea_ivarsp_bt": "Var growth pre (split tri)",
    "egrea_istdsp_bt": "Std. growth pre (split tri)",
    "eincgr_imean_bt": "Mean HH income growth pre",
    "eincgr_ivar_bt": "Var HH income growth pre",
    "eincgr_istd_bt": "STd. HH income growth pre",
    "wgt": "weight"
}

# Function to create descriptive names for wave-specific columns
def get_descriptive(col):
    for old_var, desc in name_mapping.items():
        if col.startswith(old_var + "_wave"):
            rest = col[len(old_var)+1:]  # keep wave and category suffix
            # replace underscores in the suffix with hyphens for readability
            rest = rest.replace("_", "-")
            return f"{desc}-{rest}"
        elif col == old_var:
            return desc
    return ""  # if no mapping, leave empty in second row

# Create descriptive row
descriptive_row = [get_descriptive(col) for col in df_clean.columns]

# Combine original header and descriptive row
df_with_header = pd.DataFrame([df_clean.columns.tolist(), descriptive_row] + df_clean.values.tolist())

# Save to CSV (no header, first row = original names, second row = descriptive)
df_with_header.to_csv(r"C:\Users\Zach\PycharmProjects\AppliedML\ces_gdp_rct_v1a_with_descriptive_second_row.csv",
                      index=False, header=False)

print("CSV saved with descriptive variable names in the second row!")


CSV saved with descriptive variable names in the second row!


In [2]:
import pandas as pd

# Load the cleaned dataset
df_clean = pd.read_csv(r"C:\Users\Zach\PycharmProjects\AppliedML\ces_gdp_rct_v1a_hid_clean_final_no_missing.csv")

# Convert 'hid' to string to prevent Excel rounding/scientific notation
df_clean['hid'] = df_clean['hid'].astype(str)

# Mapping of base variable names -> descriptive names
name_mapping = {
    "country": "Country",
    "pr2010": "Recruitment Type",
    "pr2110": "Method & Phe0type",
    "age": "Age",
    "b2010": "Employment Situation",
    "b3300": "Housing Type",
    "b7010": "Total Combined Net of Tax Income Option",
    "q2021": "Employment Status",
    "wave": "Questionare Wave",
    "j1020": "Industry",
    "k1010_1": "Min Expectations of Euro Area Growth",
    "k1010_2": "Max Expectations of Euro Area Growth",
    "k1010_3": "Growth uncertainty probability",
    "k2000": "Treatment Group",
    "k2020_1": "Probability of low growth rate",
    "k2020_2": "Probability of medium growth rate",
    "k2020_3": "Probability of max growth rate",
    "k2110_1": "Min Household Income Growth",
    "k2110_2": "Med Household Income Growth",
    "k2110_3": "Max Household Income Growth",
    "k2120_1": "Probabilty Min Income Growth",
    "k2120_2": "Probabilty Med Income Growth",
    "k2120_3": "Probabilty Max Income Growth",
    "treatment": "Experimental Group",
    "educ_3": "Education Categories",
    "hid": "Respondent ID",
    "hhsize": "Household Size",
    "male": "Male",
    "dedu1": "Primary School (most education)",
    "dedu2": "Secondary School",
    "dedu3": "Post-Secondary School",
    "eincgr": "Expected Net Household Income Growth in next year",
    "liquid": "Sufficient Liquidity",
    "buypm_hm": "Purchased house/apartment in past",
    "buypm_car": "Purchased car/other vehicle in past",
    "buypm_dur": "Purchased home appliance/furinture in past",
    "buypm_holid": "Purchased home holiday in past",
    "buypm_lux": "Purchased luxury good in past",
    "buypm_oth": "Purchased other major item in past",
    "buypm_0ne": "Purchased 0 major item in past",
    "pbuy_hm": "Plan to buy a home in next year",
    "pbuy_car": "Plan to buy a car in next year",
    "pbuy_dur": "Plan to buy home appliance, furniture, etc in next year",
    "pbuy_holid": "Plan to buy a holiday in next year",
    "pbuy_lux": "Plan to buy a luxury good in next year",
    "pbuy_oth": "Plan to buy other major purchase in next year",
    "pbuy_0ne": "Plan to buy a 0thing major in next year",
    "eincgr_imean": "Mean expected income growth",
    "eincgr_istd": "Standard deviation of expected income growth",
    "cons_dbt": "Debt repayments",
    "cons_tot": "Total expenditure final",
    "bs_food": "Food budget share",
    "bs_foodin": "Inside food budget share",
    "bs_foodout": "Outside food budget share",
    "bs_hous": "House budget share",
    "bs_utl": "Utilities budget share",
    "bs_hmeqp": "Home appliances budget share",
    "bs_dbt": "Debt budget share",
    "bs_cloth": "Clothes budget share",
    "bs_hlth": "Health budget share",
    "bs_transp": "Transportation budget share",
    "bs_recr": "Recreation budget share",
    "bs_edu": "Education budget share",
    "bs_oth": "Other budget share",
    "shinv_sav": "Savings asset share",
    "shinv_stock": "Stocks asset share",
    "shinv_mutf": "Mutual funds asset share",
    "shinv_ira": "Pension asset share",
    "shinv_stbond": "Short-term bonds asset share",
    "shinv_ltbond": "Long-term bonds asset share",
    "shinv_crypto": "Crypto asset share",
    "ostock": "Owns stocks",
    "omutf": "Owns mutual fund",
    "sh_sav": "Savings wealth share",
    "sh_stock": "Stock wealth share",
    "sh_mutf": "Mutual fund wealth share",
    "sh_ira": "Pension wealth share",
    "sh_bond": "Bonds wealth share",
    "sh_oth": "Other wealth share",
    "egrea_iskew_bt": "Skewness of growth",
    "egrea_iiqr_bt": "IQR",
    "egrea_imean_pt": "Mean growth post",
    "egrea_ivar_pt": "Variance growth post",
    "egrea_istd_pt": "Std growth post",
    "egrea_iskew_pt": "Skew post",
    "egrea_iiqr_pt": "IQR post",
    "eincgr_imean_pt": "Mean HH growth most",
    "eincgr_ivar_pt": "Var HH growth post",
    "eincgr_istd_pt": "Std. HH growth post",
    "cnt1": "Belgium",
    "cnt2": "Denmark",
    "cnt3": "Spain",
    "cnt4": "France",
    "cnt5": "Italy",
    "cnt6": "Netherlands",
    "treat1": "Control group",
    "treat2": "Mean treatment",
    "treat3": "Uncertainty Treatment (EU)",
    "treat4": "Both Treatment",
    "treat5": "Country Uncertainty Treatment",
    "egrea_imean_bt": "Mean growth pre",
    "egrea_ivar_bt": "Variance growth pre",
    "egrea_istd_bt": "Std. growth pre",
    "egrea_imeansp_bt": "Mean growth pre (split triangle)",
    "egrea_ivarsp_bt": "Var growth pre (split tri)",
    "egrea_istdsp_bt": "Std. growth pre (split tri)",
    "eincgr_imean_bt": "Mean HH income growth pre",
    "eincgr_ivar_bt": "Var HH income growth pre",
    "eincgr_istd_bt": "STd. HH income growth pre",
    "wgt": "weight"
}

# Function to create descriptive names for wave-specific columns
def get_descriptive(col):
    for old_var, desc in name_mapping.items():
        if col.startswith(old_var + "_wave"):
            rest = col[len(old_var)+1:]  # keep wave and category suffix
            rest = rest.replace("_", "-")  # nicer format
            return f"{desc}-{rest}"
        elif col == old_var:
            return desc
    return ""  # leave empty if no mapping

# Create descriptive row
descriptive_row = [get_descriptive(col) for col in df_clean.columns]

# Combine original header and descriptive row
df_with_header = pd.DataFrame([df_clean.columns.tolist(), descriptive_row] + df_clean.values.tolist())

# Save to CSV (no header, first row = original names, second row = descriptive)
df_with_header.to_csv(r"C:\Users\Zach\PycharmProjects\AppliedML\ces_gdp_rct_v1a_with_descriptive_second_row.csv",
                      index=False, header=False)

print("CSV saved with descriptive variable names in the second row, 'hid' as string to prevent rounding.")


CSV saved with descriptive variable names in the second row, 'hid' as string to prevent rounding.


In [34]:
import pandas as pd

# Load the cleaned dataset
df_clean = pd.read_csv(r"C:\Users\Zach\PycharmProjects\AppliedML\ces_gdp_rct_v1a_hid_clean_final_no_missing.csv")

# Ensure 'hid' column is fully string type
df_clean['hid'] = df_clean['hid'].astype(str)

# Mapping of base variable names -> descriptive names (same as before)
name_mapping = {
    "country": "Country",
    "pr2010": "Recruitment Type",
    "pr2110": "Method & Phe0type",
    "age": "Age",
    "b2010": "Employment Situation",
    "b3300": "Housing Type",
    "b7010": "Total Combined Net of Tax Income Option",
    "q2021": "Employment Status",
    "wave": "Questionare Wave",
    "j1020": "Industry",
    "k1010_1": "Min Expectations of Euro Area Growth",
    "k1010_2": "Max Expectations of Euro Area Growth",
    "k1010_3": "Growth uncertainty probability",
    "k2000": "Treatment Group",
    "k2020_1": "Probability of low growth rate",
    "k2020_2": "Probability of medium growth rate",
    "k2020_3": "Probability of max growth rate",
    "k2110_1": "Min Household Income Growth",
    "k2110_2": "Med Household Income Growth",
    "k2110_3": "Max Household Income Growth",
    "k2120_1": "Probabilty Min Income Growth",
    "k2120_2": "Probabilty Med Income Growth",
    "k2120_3": "Probabilty Max Income Growth",
    "treatment": "Experimental Group",
    "educ_3": "Education Categories",
    "hid": "Respondent ID",
    # ... add the rest of your mappings here
    "wgt": "weight"
}

# Function to create descriptive names for wave-specific columns
def get_descriptive(col):
    for old_var, desc in name_mapping.items():
        if col.startswith(old_var + "_wave"):
            rest = col[len(old_var)+1:]  # keep wave and category suffix
            rest = rest.replace("_", "-")  # nicer format
            return f"{desc}-{rest}"
        elif col == old_var:
            return desc
    return ""  # leave empty if no mapping

# Create descriptive row
descriptive_row = [get_descriptive(col) for col in df_clean.columns]

# Combine original header and descriptive row
df_with_header = pd.DataFrame([df_clean.columns.tolist(), descriptive_row] + df_clean.values.tolist())

# Save to CSV with 'hid' as string to prevent rounding
df_with_header.to_csv(r"C:\Users\Zach\PycharmProjects\AppliedML\ces_gdp_rct_v1a_with_descriptive_second_row.csv",
                      index=False, header=False)

print("CSV saved with 'hid' fully as strings and descriptive variable names in second row.")


CSV saved with 'hid' fully as strings and descriptive variable names in second row.


In [35]:
import pandas as pd

# Path to your original Stata file
stata_file = r"C:\Users\Zach\PycharmProjects\AppliedML\ces_gdp_rct_v1a.dta"

# Load the Stata dataset
df = pd.read_stata(stata_file)

# Optional: ensure any identifier columns (like 'hid') are strings
if 'hid' in df.columns:
    df['hid'] = df['hid'].astype(str)

# Path to save the CSV
csv_file = r"C:\Users\Zach\PycharmProjects\AppliedML\ces_gdp_rct_v1a.csv"

# Save to CSV
df.to_csv(csv_file, index=False)

print(f"Stata file successfully converted to CSV: {csv_file}")


Stata file successfully converted to CSV: C:\Users\Zach\PycharmProjects\AppliedML\ces_gdp_rct_v1a.csv


In [6]:
!pip install pingouin

  Obtaining dependency information for pingouin from https://files.pythonhosted.org/packages/eb/56/6d3607f3a78aee1de8e5466f5171722c8e344266a12dc44ccb73d024b3b3/pingouin-0.5.5-py3-none-any.whl.metadata
  Obtaining dependency information for pandas-flavor from https://files.pythonhosted.org/packages/5d/e6/71ed4d95676098159b533c4a4c424cf453fec9614edaff1a0633fe228eef/pandas_flavor-0.7.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/204.4 kB ? eta -:--:--
   ------------ --------------------------- 61.4/204.4 kB 1.7 MB/s eta 0:00:01
   -------------------------------------- - 194.6/204.4 kB 3.0 MB/s eta 0:00:01
   ---------------------------------------- 204.4/204.4 kB 2.1 MB/s eta 0:00:00


In [12]:
import pandas as pd
import numpy as np
from scipy import stats

# --- 1. Load CSV (skip descriptive row) ---
df = pd.read_csv(
    r"C:\Users\Zach\PycharmProjects\AppliedML\CesDataClean.csv",
    header=0,        # first row = column names
    skiprows=[1],    # skip second row (descriptive names)
    dtype={'hid': str}  # ensure HID is string
)
df['ID'] = df['hid']

# --- 2. Convert key numeric columns safely ---
numeric_cols = ['b7010', 'cons_tot', 'cons_dbt', 'age', 'hhsize', 'male', 'b2010', 'b3300', 'k2000', 'wgt']
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# --- 3. Winsorize function ---
def winsorize(series, lower=1, upper=99):
    series = pd.to_numeric(series, errors='coerce')
    lower_val = np.percentile(series.dropna(), lower)
    upper_val = np.percentile(series.dropna(), upper)
    return series.clip(lower_val, upper_val)

# --- 4. Transform variables ---
df['income0_winsorized'] = winsorize(df['b7010']) / 1000
df['nondur'] = df['cons_tot'] - df['cons_dbt']
df['nondur_w'] = winsorize(df['nondur'])
df['nondur_w_L2'] = df.groupby('hid')['nondur_w'].shift(2) / 1000

# --- 5. Create dummy variables ---
for cat in range(1, 7):
    df[f'b2010_v{cat}'] = (df['b2010'] == cat).astype(int)
for cat in range(1, 4):
    df[f'b3300_v{cat}'] = (df['b3300'] == cat).astype(int)

# --- 6. Weighted statistics ---
cont_vars = ['age', 'hhsize', 'income0_winsorized', 'nondur_w_L2']
binary_vars = ['male'] + [f'b2010_v{cat}' for cat in range(1,7)] + [f'b3300_v{cat}' for cat in range(1,4)]
weight_var = 'wgt'
treat_var = 'k2000'

weighted_stats = {}
pvals = {}

for var in cont_vars + binary_vars:
    grouped = df.groupby(treat_var).apply(lambda x: pd.Series({
        'mean': np.average(x[var].dropna(), weights=x[weight_var].loc[x[var].notna()]) if x[var].notna().any() else np.nan,
        'std': np.sqrt(np.cov(x[var].dropna(), aweights=x[weight_var].loc[x[var].notna()])) if x[var].notna().any() else np.nan
    }))
    overall_mean = np.average(df[var].dropna(), weights=df[weight_var].loc[df[var].notna()]) if df[var].notna().any() else np.nan
    overall_std = np.sqrt(np.cov(df[var].dropna(), aweights=df[weight_var].loc[df[var].notna()])) if df[var].notna().any() else np.nan
    weighted_stats[var] = {'grouped': grouped, 'overall_mean': overall_mean, 'overall_std': overall_std}

    # Optional p-value using unweighted ANOVA
    try:
        groups = [df[df[treat_var]==k][var].dropna() for k in df[treat_var].unique()]
        _, pval = stats.f_oneway(*groups)
    except:
        pval = np.nan
    pvals[var] = pval

# --- 7. Number of observations ---
nobs = df.groupby(treat_var).size()
nobs_all = df.shape[0]

# --- 8. Build Table01 ---
table_rows = []
for var, stats_dict in weighted_stats.items():
    row_mean = [var + ' mean'] + stats_dict['grouped']['mean'].tolist() + [stats_dict['overall_mean'], pvals.get(var, np.nan)]
    row_std = [var + ' std'] + stats_dict['grouped']['std'].tolist() + [stats_dict['overall_std'], np.nan]
    table_rows.append(row_mean)
    table_rows.append(row_std)

# Observation counts
row_nobs = ['Nobs'] + [nobs.get(i, np.nan) for i in range(1,6)] + [nobs_all, np.nan]
table_rows.append(row_nobs)

table_df = pd.DataFrame(table_rows)

# --- 9. Export Table01 ---
table_df.to_csv(r"C:\Users\Zach\PycharmProjects\AppliedML\Table01_CesDataClean.csv",
                index=False, header=False)

print("Table01 generated from CSV, descriptive row skipped, numeric transformations safe.")



KeyError: 'cons_tot'

In [16]:
import pandas as pd

# Load CSV and skip descriptive row (second row)
df = pd.read_csv(
    r"C:\Users\Zach\PycharmProjects\AppliedML\CesDataClean.csv",
    header=0,         # first row is actual variable names
    skiprows=[1],     # skip the descriptive row
    dtype={'hid': str} # ensure hid is string
)

# Strip spaces and remove duplicate columns if any
df.columns = df.columns.str.strip()
df = df.loc[:, ~df.columns.duplicated()]

# Check that 'hid' is 1D
print(df['hid'].shape)
print(df['hid'].head())


(3309,)
0    130200004550
1    130200023550
2    130200033550
3    130200035550
4    130200044550
Name: hid, dtype: object


In [18]:
import pandas as pd
import numpy as np
from scipy import stats

# --- 1. Load CSV, skip descriptive row ---
df = pd.read_csv(
    r"C:\Users\Zach\PycharmProjects\AppliedML\CesDataClean.csv",
    header=0,        # first row is variable names
    skiprows=[1],    # skip descriptive row
    dtype={'hid': str}  # ensure HID is string
)
df.columns = df.columns.str.strip()
df = df.loc[:, ~df.columns.duplicated()]

# --- 2. Keep allowed variable prefixes ---
allowed_prefixes = ['country', 'pr2010', 'pr2110', 'age', 'b2010', 'b3300', 'b7010', 'q2021', 'wave',
                    'j1020', 'k1010_1', 'k1010_2', 'k1010_3', 'k2000', 'k2020_1', 'k2020_2', 'k2020_3',
                    'k2110_1', 'k2110_2', 'k2110_3', 'k2120_1', 'k2120_2', 'k2120_3', 'treatment', 
                    'educ_3', 'hid', 'hhsize', 'male', 'dedu1', 'dedu2', 'dedu3', 'lhhnetinc', 'eincgr', 
                    'liquid','buypm_hm','buypm_car','buypm_dur','buypm_holid','buypm_lux','buypm_oth',
                    'buypm_none','pbuy_hm','pbuy_car','pbuy_dur','pbuy_holid','pbuy_lux','pbuy_oth',
                    'pbuy_none','eincgr_imean','eincgr_istd','cons_dbt','cons_tot','bs_food','bs_foodin',
                    'bs_foodout','bs_hous','bs_utl','bs_hmeqp','bs_dbt','bs_cloth','bs_hlth','bs_transp',
                    'bs_recr','bs_edu','bs_oth','shinv_sav','shinv_stock','shinv_mutf','shinv_ira',
                    'shinv_stbond','shinv_ltbond','shinv_crypto','ostock','omutf','sh_sav','sh_stock',
                    'sh_mutf','sh_ira','sh_bond','sh_oth','egrea_iskew_bt','egrea_iiqr_bt','egrea_imean_pt',
                    'egrea_ivar_pt','egrea_istd_pt','egrea_iskew_pt','egrea_iiqr_pt','egrea_iqr_pt_flag',
                    'eincgr_imean_pt','eincgr_ivar_pt','eincgr_istd_pt','cnt1','cnt2','cnt3','cnt4','cnt5',
                    'cnt6','egrea_imean_pt_old','egrea_ivar_pt_old','egrea_istd_pt_old','treat1','treat2',
                    'treat3','treat4','treat5','egrea_imean_bt','egrea_ivar_bt','egrea_istd_bt',
                    'egrea_imeansp_bt','egrea_ivarsp_bt','egrea_istdsp_bt','eincgr_imean_bt','eincgr_ivar_bt',
                    'eincgr_istd_bt','flag_speeder','wgt']

df = df[[col for col in df.columns if any(col.startswith(p) for p in allowed_prefixes)]]

# --- 3. Ensure HID is string ---
df['hid'] = df['hid'].astype(str)

# --- 4. Detect numeric vs categorical variables ---
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
numeric_cols = [c for c in numeric_cols if c != 'wgt']  # exclude weight
categorical_cols = [c for c in df.columns if c not in numeric_cols + ['hid', 'wgt']]

# --- 5. Winsorize numeric variables and create 2-period lag ---
def winsorize(series, lower=1, upper=99):
    series = pd.to_numeric(series, errors='coerce')
    lower_val = np.percentile(series.dropna(), lower)
    upper_val = np.percentile(series.dropna(), upper)
    return series.clip(lower_val, upper_val)

for col in numeric_cols:
    df[f'{col}_w'] = winsorize(df[col])
    df[f'{col}_w_L2'] = df.groupby('hid')[f'{col}_w'].shift(2)

# --- 6. Convert categorical variables to dummies ---
for col in categorical_cols:
    df[col] = df[col].astype(str)
    for val in df[col].dropna().unique():
        df[f'{col}-wave{val}'] = (df[col] == val).astype(int)

# --- 7. Detect treatment variable dynamically ---
treat_cols = [c for c in df.columns if c.startswith('k2000')]
if not treat_cols:
    raise ValueError("No treatment column starting with 'k2000' found.")
treat_var = treat_cols[0]
print("Using treatment variable:", treat_var)

# --- 8. Compute weighted stats and ANOVA p-values ---
weight_var = 'wgt'
cont_vars = [f'{col}_w_L2' for col in numeric_cols]
binary_vars = [c for c in df.columns if c not in cont_vars + ['hid', weight_var, treat_var]]

weighted_stats = {}
pvals = {}

for var in cont_vars + binary_vars:
    grouped = df.groupby(treat_var).apply(
        lambda x: pd.Series({
            'mean': np.average(x[var].dropna(), weights=x[weight_var].loc[x[var].notna()]) 
                    if x[var].notna().any() else np.nan,
            'std': np.sqrt(np.cov(x[var].dropna(), aweights=x[weight_var].loc[x[var].notna()])) 
                    if x[var].notna().any() else np.nan
        })
    )
    overall_mean = np.average(df[var].dropna(), weights=df[weight_var].loc[df[var].notna()]) if df[var].notna().any() else np.nan
    overall_std = np.sqrt(np.cov(df[var].dropna(), aweights=df[weight_var].loc[df[var].notna()])) if df[var].notna().any() else np.nan
    weighted_stats[var] = {'grouped': grouped, 'overall_mean': overall_mean, 'overall_std': overall_std}

    # ANOVA p-value
    try:
        groups = [df[df[treat_var]==k][var].dropna() for k in df[treat_var].unique()]
        _, pval = stats.f_oneway(*groups)
    except:
        pval = np.nan
    pvals[var] = pval

# --- 9. Observation counts ---
nobs = df.groupby(treat_var).size()
nobs_all = df.shape[0]

# --- 10. Build Table01 ---
table_rows = []
for var, stats_dict in weighted_stats.items():
    row_mean = [var + ' mean'] + stats_dict['grouped']['mean'].tolist() + [stats_dict['overall_mean'], pvals.get(var, np.nan)]
    row_std = [var + ' std'] + stats_dict['grouped']['std'].tolist() + [stats_dict['overall_std'], np.nan]
    table_rows.append(row_mean)
    table_rows.append(row_std)

row_nobs = ['Nobs'] + [nobs.get(i, np.nan) for i in range(1,6)] + [nobs_all, np.nan]
table_rows.append(row_nobs)

table_df = pd.DataFrame(table_rows)

# --- 11. Export Table01 ---
table_df.to_csv(r"C:\Users\Zach\PycharmProjects\AppliedML\Table01_CesDataClean.csv", index=False, header=False)

print("✅ Table01 successfully generated.")



C:\Users\Zach\AppData\Local\Temp\ipykernel_35592\1946136640.py:53: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_w_L2'] = df.groupby('hid')[f'{col}_w'].shift(2)
C:\Users\Zach\AppData\Local\Temp\ipykernel_35592\1946136640.py:52: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_w'] = winsorize(df[col])
C:\Users\Zach\AppData\Local\Temp\ipykernel_35592\1946136640.py:53: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance. 

C:\Users\Zach\AppData\Local\Temp\ipykernel_35592\1946136640.py:53: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_w_L2'] = df.groupby('hid')[f'{col}_w'].shift(2)
C:\Users\Zach\AppData\Local\Temp\ipykernel_35592\1946136640.py:52: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_w'] = winsorize(df[col])
C:\Users\Zach\AppData\Local\Temp\ipykernel_35592\1946136640.py:53: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance. 

C:\Users\Zach\AppData\Local\Temp\ipykernel_35592\1946136640.py:53: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_w_L2'] = df.groupby('hid')[f'{col}_w'].shift(2)
C:\Users\Zach\AppData\Local\Temp\ipykernel_35592\1946136640.py:52: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_w'] = winsorize(df[col])
C:\Users\Zach\AppData\Local\Temp\ipykernel_35592\1946136640.py:53: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance. 

C:\Users\Zach\AppData\Local\Temp\ipykernel_35592\1946136640.py:52: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_w'] = winsorize(df[col])
C:\Users\Zach\AppData\Local\Temp\ipykernel_35592\1946136640.py:53: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_w_L2'] = df.groupby('hid')[f'{col}_w'].shift(2)
C:\Users\Zach\AppData\Local\Temp\ipykernel_35592\1946136640.py:52: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance. 

C:\Users\Zach\AppData\Local\Temp\ipykernel_35592\1946136640.py:53: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_w_L2'] = df.groupby('hid')[f'{col}_w'].shift(2)
C:\Users\Zach\AppData\Local\Temp\ipykernel_35592\1946136640.py:52: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_w'] = winsorize(df[col])
C:\Users\Zach\AppData\Local\Temp\ipykernel_35592\1946136640.py:53: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance. 

...

C:\Users\Zach\anaconda3\Lib\site-packages\scipy\stats\_stats_py.py:4133: DegenerateDataWarning: at least one input has length 0
  warnings.warn(stats.DegenerateDataWarning('at least one input '
C:\Users\Zach\anaconda3\Lib\site-packages\scipy\stats\_stats_py.py:4133: DegenerateDataWarning: at least one input has length 0
  warnings.warn(stats.DegenerateDataWarning('at least one input '
C:\Users\Zach\anaconda3\Lib\site-packages\scipy\stats\_stats_py.py:4133: DegenerateDataWarning: at least one input has length 0
  warnings.warn(stats.DegenerateDataWarning('at least one input '
C:\Users\Zach\anaconda3\Lib\site-packages\scipy\stats\_stats_py.py:4133: DegenerateDataWarning: at least one input has length 0
  warnings.warn(stats.DegenerateDataWarning('at least one input '
C:\Users\Zach\anaconda3\Lib\site-packages\scipy\stats\_stats_py.py:4133: DegenerateDataWarning: at least one input has length 0
  warnings.warn(stats.DegenerateDataWarning('at least one input '
C:\Users\Zach\anaconda3\Lib\si

C:\Users\Zach\anaconda3\Lib\site-packages\scipy\stats\_stats_py.py:4133: DegenerateDataWarning: at least one input has length 0
  warnings.warn(stats.DegenerateDataWarning('at least one input '
C:\Users\Zach\anaconda3\Lib\site-packages\scipy\stats\_stats_py.py:4133: DegenerateDataWarning: at least one input has length 0
  warnings.warn(stats.DegenerateDataWarning('at least one input '
C:\Users\Zach\anaconda3\Lib\site-packages\scipy\stats\_stats_py.py:4133: DegenerateDataWarning: at least one input has length 0
  warnings.warn(stats.DegenerateDataWarning('at least one input '
C:\Users\Zach\anaconda3\Lib\site-packages\scipy\stats\_stats_py.py:4133: DegenerateDataWarning: at least one input has length 0
  warnings.warn(stats.DegenerateDataWarning('at least one input '
C:\Users\Zach\anaconda3\Lib\site-packages\scipy\stats\_stats_py.py:4133: DegenerateDataWarning: at least one input has length 0
  warnings.warn(stats.DegenerateDataWarning('at least one input '
C:\Users\Zach\anaconda3\Lib\si

C:\Users\Zach\anaconda3\Lib\site-packages\scipy\stats\_stats_py.py:4133: DegenerateDataWarning: at least one input has length 0
  warnings.warn(stats.DegenerateDataWarning('at least one input '
C:\Users\Zach\anaconda3\Lib\site-packages\scipy\stats\_stats_py.py:4133: DegenerateDataWarning: at least one input has length 0
  warnings.warn(stats.DegenerateDataWarning('at least one input '
C:\Users\Zach\anaconda3\Lib\site-packages\scipy\stats\_stats_py.py:4133: DegenerateDataWarning: at least one input has length 0
  warnings.warn(stats.DegenerateDataWarning('at least one input '
C:\Users\Zach\anaconda3\Lib\site-packages\scipy\stats\_stats_py.py:4133: DegenerateDataWarning: at least one input has length 0
  warnings.warn(stats.DegenerateDataWarning('at least one input '
C:\Users\Zach\anaconda3\Lib\site-packages\scipy\stats\_stats_py.py:4133: DegenerateDataWarning: at least one input has length 0
  warnings.warn(stats.DegenerateDataWarning('at least one input '
C:\Users\Zach\anaconda3\Lib\si

C:\Users\Zach\anaconda3\Lib\site-packages\scipy\stats\_stats_py.py:4133: DegenerateDataWarning: at least one input has length 0
  warnings.warn(stats.DegenerateDataWarning('at least one input '
C:\Users\Zach\anaconda3\Lib\site-packages\scipy\stats\_stats_py.py:4133: DegenerateDataWarning: at least one input has length 0
  warnings.warn(stats.DegenerateDataWarning('at least one input '
C:\Users\Zach\anaconda3\Lib\site-packages\scipy\stats\_stats_py.py:4133: DegenerateDataWarning: at least one input has length 0
  warnings.warn(stats.DegenerateDataWarning('at least one input '
C:\Users\Zach\anaconda3\Lib\site-packages\scipy\stats\_stats_py.py:4133: DegenerateDataWarning: at least one input has length 0
  warnings.warn(stats.DegenerateDataWarning('at least one input '
C:\Users\Zach\anaconda3\Lib\site-packages\scipy\stats\_stats_py.py:4133: DegenerateDataWarning: at least one input has length 0
  warnings.warn(stats.DegenerateDataWarning('at least one input '
C:\Users\Zach\anaconda3\Lib\si

KeyError: 'wgt'

In [20]:
import pandas as pd
import numpy as np
from scipy import stats

# --- 1. Load CSV, skip descriptive row ---
df = pd.read_csv(
    r"C:\Users\Zach\PycharmProjects\AppliedML\CesDataClean.csv",
    header=0,        # first row is variable names
    skiprows=[1],    # skip descriptive row
    dtype={'hid': str}  # ensure HID is string
)
df.columns = df.columns.str.strip()
df = df.loc[:, ~df.columns.duplicated()]

# --- 2. Ensure HID is string ---
df['hid'] = df['hid'].astype(str)

# --- 3. Detect treatment variable dynamically ---
treat_cols = [c for c in df.columns if c.startswith('k2000')]
if not treat_cols:
    raise ValueError("No treatment column starting with 'k2000' found.")
treat_var = treat_cols[0]
print("Using treatment variable:", treat_var)

# --- 4. Detect weight variable dynamically ---
weight_cols = [c for c in df.columns if c.startswith('wgt')]
if not weight_cols:
    raise ValueError("No weight column starting with 'wgt' found.")
weight_var = weight_cols[0]
print("Using weight variable:", weight_var)

# --- 5. Detect numeric vs categorical variables dynamically ---
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
numeric_cols = [c for c in numeric_cols if c not in [weight_var]]  # exclude weight
categorical_cols = [c for c in df.columns if c not in numeric_cols + ['hid', treat_var, weight_var]]

# --- 6. Winsorize numeric variables and create 2-period lag ---
def winsorize(series, lower=1, upper=99):
    series = pd.to_numeric(series, errors='coerce')
    lower_val = np.percentile(series.dropna(), lower)
    upper_val = np.percentile(series.dropna(), upper)
    return series.clip(lower_val, upper_val)

for col in numeric_cols:
    df[f'{col}_w'] = winsorize(df[col])
    df[f'{col}_w_L2'] = df.groupby('hid')[f'{col}_w'].shift(2)

# --- 7. Convert categorical variables to dummies dynamically ---
for col in categorical_cols:
    df[col] = df[col].astype(str)
    for val in df[col].dropna().unique():
        df[f'{col}-wave{val}'] = (df[col] == val).astype(int)

# --- 8. Compute weighted stats and ANOVA p-values ---
cont_vars = [f'{col}_w_L2' for col in numeric_cols]
binary_vars = [c for c in df.columns if c not in cont_vars + ['hid', weight_var, treat_var]]

weighted_stats = {}
pvals = {}

for var in cont_vars + binary_vars:
    grouped = df.groupby(treat_var).apply(
        lambda x: pd.Series({
            'mean': np.average(x[var].dropna(), weights=x[weight_var].loc[x[var].notna()]) 
                    if x[var].notna().any() else np.nan,
            'std': np.sqrt(np.cov(x[var].dropna(), aweights=x[weight_var].loc[x[var].notna()])) 
                    if x[var].notna().any() else np.nan
        })
    )
    overall_mean = np.average(df[var].dropna(), weights=df[weight_var].loc[df[var].notna()]) if df[var].notna().any() else np.nan
    overall_std = np.sqrt(np.cov(df[var].dropna(), aweights=df[weight_var].loc[df[var].notna()])) if df[var].notna().any() else np.nan
    weighted_stats[var] = {'grouped': grouped, 'overall_mean': overall_mean, 'overall_std': overall_std}

    # ANOVA p-value
    try:
        groups = [df[df[treat_var]==k][var].dropna() for k in df[treat_var].unique()]
        _, pval = stats.f_oneway(*groups)
    except:
        pval = np.nan
    pvals[var] = pval

# --- 9. Observation counts ---
nobs = df.groupby(treat_var).size()
nobs_all = df.shape[0]

# --- 10. Build Table01 ---
table_rows = []
for var, stats_dict in weighted_stats.items():
    row_mean = [var + ' mean'] + stats_dict['grouped']['mean'].tolist() + [stats_dict['overall_mean'], pvals.get(var, np.nan)]
    row_std = [var + ' std'] + stats_dict['grouped']['std'].tolist() + [stats_dict['overall_std'], np.nan]
    table_rows.append(row_mean)
    table_rows.append(row_std)

row_nobs = ['Nobs'] + [nobs.get(i, np.nan) for i in sorted(df[treat_var].unique())] + [nobs_all, np.nan]
table_rows.append(row_nobs)

table_df = pd.DataFrame(table_rows)

# --- 11. Export Table01 ---
table_df.to_csv(r"C:\Users\Zach\PycharmProjects\AppliedML\Table01_CesDataClean.csv", index=False, header=False)

print("✅ Table01 successfully generated with dynamic variable detection.")


Using treatment variable: k2000_Control
Using weight variable: wgt_wave7


C:\Users\Zach\AppData\Local\Temp\ipykernel_35592\2949367654.py:46: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_w_L2'] = df.groupby('hid')[f'{col}_w'].shift(2)
C:\Users\Zach\AppData\Local\Temp\ipykernel_35592\2949367654.py:45: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_w'] = winsorize(df[col])
C:\Users\Zach\AppData\Local\Temp\ipykernel_35592\2949367654.py:46: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance. 

C:\Users\Zach\AppData\Local\Temp\ipykernel_35592\2949367654.py:46: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_w_L2'] = df.groupby('hid')[f'{col}_w'].shift(2)
C:\Users\Zach\AppData\Local\Temp\ipykernel_35592\2949367654.py:45: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_w'] = winsorize(df[col])
C:\Users\Zach\AppData\Local\Temp\ipykernel_35592\2949367654.py:46: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance. 

C:\Users\Zach\AppData\Local\Temp\ipykernel_35592\2949367654.py:45: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_w'] = winsorize(df[col])
C:\Users\Zach\AppData\Local\Temp\ipykernel_35592\2949367654.py:46: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_w_L2'] = df.groupby('hid')[f'{col}_w'].shift(2)
C:\Users\Zach\AppData\Local\Temp\ipykernel_35592\2949367654.py:45: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance. 

C:\Users\Zach\AppData\Local\Temp\ipykernel_35592\2949367654.py:46: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_w_L2'] = df.groupby('hid')[f'{col}_w'].shift(2)
C:\Users\Zach\AppData\Local\Temp\ipykernel_35592\2949367654.py:45: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_w'] = winsorize(df[col])
C:\Users\Zach\AppData\Local\Temp\ipykernel_35592\2949367654.py:46: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance. 

...

C:\Users\Zach\anaconda3\Lib\site-packages\scipy\stats\_stats_py.py:4133: DegenerateDataWarning: at least one input has length 0
  warnings.warn(stats.DegenerateDataWarning('at least one input '
C:\Users\Zach\anaconda3\Lib\site-packages\scipy\stats\_stats_py.py:4133: DegenerateDataWarning: at least one input has length 0
  warnings.warn(stats.DegenerateDataWarning('at least one input '
C:\Users\Zach\anaconda3\Lib\site-packages\scipy\stats\_stats_py.py:4133: DegenerateDataWarning: at least one input has length 0
  warnings.warn(stats.DegenerateDataWarning('at least one input '
C:\Users\Zach\anaconda3\Lib\site-packages\scipy\stats\_stats_py.py:4133: DegenerateDataWarning: at least one input has length 0
  warnings.warn(stats.DegenerateDataWarning('at least one input '
C:\Users\Zach\anaconda3\Lib\site-packages\scipy\stats\_stats_py.py:4133: DegenerateDataWarning: at least one input has length 0
  warnings.warn(stats.DegenerateDataWarning('at least one input '
C:\Users\Zach\anaconda3\Lib\si

C:\Users\Zach\anaconda3\Lib\site-packages\scipy\stats\_stats_py.py:4133: DegenerateDataWarning: at least one input has length 0
  warnings.warn(stats.DegenerateDataWarning('at least one input '
C:\Users\Zach\anaconda3\Lib\site-packages\scipy\stats\_stats_py.py:4133: DegenerateDataWarning: at least one input has length 0
  warnings.warn(stats.DegenerateDataWarning('at least one input '
C:\Users\Zach\anaconda3\Lib\site-packages\scipy\stats\_stats_py.py:4133: DegenerateDataWarning: at least one input has length 0
  warnings.warn(stats.DegenerateDataWarning('at least one input '
C:\Users\Zach\anaconda3\Lib\site-packages\scipy\stats\_stats_py.py:4133: DegenerateDataWarning: at least one input has length 0
  warnings.warn(stats.DegenerateDataWarning('at least one input '
C:\Users\Zach\anaconda3\Lib\site-packages\scipy\stats\_stats_py.py:4133: DegenerateDataWarning: at least one input has length 0
  warnings.warn(stats.DegenerateDataWarning('at least one input '
C:\Users\Zach\anaconda3\Lib\si

C:\Users\Zach\anaconda3\Lib\site-packages\scipy\stats\_stats_py.py:4133: DegenerateDataWarning: at least one input has length 0
  warnings.warn(stats.DegenerateDataWarning('at least one input '
C:\Users\Zach\anaconda3\Lib\site-packages\scipy\stats\_stats_py.py:4133: DegenerateDataWarning: at least one input has length 0
  warnings.warn(stats.DegenerateDataWarning('at least one input '
C:\Users\Zach\anaconda3\Lib\site-packages\scipy\stats\_stats_py.py:4133: DegenerateDataWarning: at least one input has length 0
  warnings.warn(stats.DegenerateDataWarning('at least one input '
C:\Users\Zach\anaconda3\Lib\site-packages\scipy\stats\_stats_py.py:4133: DegenerateDataWarning: at least one input has length 0
  warnings.warn(stats.DegenerateDataWarning('at least one input '
C:\Users\Zach\anaconda3\Lib\site-packages\scipy\stats\_stats_py.py:4133: DegenerateDataWarning: at least one input has length 0
  warnings.warn(stats.DegenerateDataWarning('at least one input '
C:\Users\Zach\anaconda3\Lib\si

C:\Users\Zach\anaconda3\Lib\site-packages\scipy\stats\_stats_py.py:4133: DegenerateDataWarning: at least one input has length 0
  warnings.warn(stats.DegenerateDataWarning('at least one input '
C:\Users\Zach\anaconda3\Lib\site-packages\scipy\stats\_stats_py.py:4133: DegenerateDataWarning: at least one input has length 0
  warnings.warn(stats.DegenerateDataWarning('at least one input '
C:\Users\Zach\anaconda3\Lib\site-packages\scipy\stats\_stats_py.py:4133: DegenerateDataWarning: at least one input has length 0
  warnings.warn(stats.DegenerateDataWarning('at least one input '
C:\Users\Zach\anaconda3\Lib\site-packages\scipy\stats\_stats_py.py:4133: DegenerateDataWarning: at least one input has length 0
  warnings.warn(stats.DegenerateDataWarning('at least one input '
C:\Users\Zach\anaconda3\Lib\site-packages\scipy\stats\_stats_py.py:4133: DegenerateDataWarning: at least one input has length 0
  warnings.warn(stats.DegenerateDataWarning('at least one input '
C:\Users\Zach\anaconda3\Lib\si

✅ Table01 successfully generated with dynamic variable detection.


In [3]:
import pandas as pd

# Load your CSV
df = pd.read_csv("CESDataClean.csv")

# Print all variable (column) names
print(df.columns.tolist())


C:\Users\Zach\AppData\Local\Temp\ipykernel_19828\3714863387.py:4: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,651,652,653,654,655,656,657,658,659,660,672,683,697,698,699,705,706,707) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("CESDataCle

['hid', 'q2021_wave7_In retirement or early retirement', 'q2021_wave7_Looking after children or other persons, doing housework', 'q2021_wave7_On extended leave (disability, sick, maternity or other leave)', 'q2021_wave7_Other', 'q2021_wave7_Studying, at school, or in training', 'q2021_wave7_Temporarily laid-off (you expect to return to your previous workplace)', 'q2021_wave7_Unable to work because of disability or other medical reasons', 'q2021_wave7_Unemployed and actively looking for a job', 'q2021_wave7_Unemployed, interested in having a job but not actively looking for a job', 'q2021_wave7_Working full-time (self-employed or working for someone else)', 'q2021_wave7_Working part-time (self-employed or working for someone else)', 'q2021_wave8_In retirement or early retirement', 'q2021_wave8_Looking after children or other persons, doing housework', 'q2021_wave8_On extended leave (disability, sick, maternity or other leave)', 'q2021_wave8_Other', 'q2021_wave8_Studying, at school, or i

In [9]:
import pandas as pd
import statsmodels.api as sm

# === Load your data ===
df = pd.read_csv(r"C:\Users\Zach\PycharmProjects\AppliedML\CesDataClean.csv")

# === Variables to summarize ===
vars_to_summarize = [
    'age', 'hhsize', 'male',
    'k2110_2_wave7', 'cons_tot_wave7',
    'dedu1', 'dedu2', 'dedu3',
    'q2021_wave7_Temporarily laid-off (you expect to return to your previous workplace)',
    'q2021_wave7_Unemployed and actively looking for a job',
    'q2021_wave7_Unemployed, interested in having a job but not actively looking for a job',
    'q2021_wave7_Working full-time (self-employed or working for someone else)',
    'q2021_wave7_Working part-time (self-employed or working for someone else)',
    'cnt1','cnt2','cnt3','cnt4','cnt5','cnt6',
    'b3300_Owner-occupied property with mortgage outstanding',
    'b3300_Owner-occupied property without mortgage outstanding',
    'b3300_Rented house/flat'
]

# === Treatment dummies ===
treatments = ['treat1','treat2','treat3','treat4','treat5']

# Convert to numeric where possible
for v in vars_to_summarize + treatments:
    if v in df.columns:
        df[v] = pd.to_numeric(df[v], errors="coerce")

# === Build results table ===
rows = []
for var in vars_to_summarize:
    if var not in df.columns:
        print(f"⚠️ Skipping {var} (not in dataset)")
        continue
    
    row_mean = {"Variable": f"{var} (Mean)"}
    row_std  = {"Variable": f"{var} (Std)"}
    
    # Loop over treatments
    for t in treatments:
        mask = df[t] == 1
        row_mean[t] = df.loc[mask, var].mean()
        row_std[t]  = df.loc[mask, var].std()
    
    # Overall
    row_mean["All"] = df[var].mean()
    row_std["All"]  = df[var].std()
    
    # p-value from F-test: does var differ across treatment groups?
    try:
        X = df[treatments]
        y = df[var]
        model = sm.OLS(y, sm.add_constant(X), missing="drop").fit()
        f_test = model.f_test("treat1 = treat2 = treat3 = treat4 = treat5 = 0")
        pval = f_test.pvalue.item()
    except Exception:
        pval = None
    
    row_mean["p-value"] = pval
    row_std["p-value"] = None  # Only show p-value on mean row
    
    # N row
    row_n = {"Variable": f"{var} (N)"}
    for t in treatments:
        row_n[t] = df.loc[df[t]==1, var].count()
    row_n["All"] = df[var].count()
    row_n["p-value"] = None
    
    rows.extend([row_mean, row_std, row_n])

# Assemble final table
table = pd.DataFrame(rows)

# === Save output ===
outpath = r"C:\Users\Zach\PycharmProjects\AppliedML\TreatmentBalanceTable.csv"
table.to_csv(outpath, index=False)

print("✅ Saved summary table to:", outpath)


C:\Users\Zach\AppData\Local\Temp\ipykernel_19828\2928102108.py:5: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,651,652,653,654,655,656,657,658,659,660,672,683,697,698,699,705,706,707) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r"C:\Users\

✅ Saved summary table to: C:\Users\Zach\PycharmProjects\AppliedML\TreatmentBalanceTable.csv


In [10]:
import pandas as pd

# Load the balance table you already created
table = pd.read_csv(r"C:\Users\Zach\PycharmProjects\AppliedML\TreatmentBalanceTable.csv")

# Round numbers for nicer display
def fmt(x):
    if pd.isna(x):
        return ""
    if isinstance(x, (int, float)):
        return f"{x:.2f}"
    return str(x)

formatted = table.copy()
for col in formatted.columns:
    if col != "Variable":
        formatted[col] = formatted[col].apply(fmt)

# Save LaTeX output
latex_table = formatted.to_latex(
    index=False,
    column_format="lccccccc",  # left + 6 centered columns
    escape=False,
    caption="Balance Table by Treatment Group",
    label="tab:balance"
)

outpath = r"C:\Users\Zach\PycharmProjects\AppliedML\TreatmentBalanceTable.tex"
with open(outpath, "w", encoding="utf-8") as f:
    f.write(latex_table)

print("✅ LaTeX table saved to:", outpath)


✅ LaTeX table saved to: C:\Users\Zach\PycharmProjects\AppliedML\TreatmentBalanceTable.tex


In [11]:
import pandas as pd
import statsmodels.api as sm

# === Load your data ===
df = pd.read_csv(r"C:\Users\Zach\PycharmProjects\AppliedML\CesDataClean.csv")

# === Variables to summarize ===
vars_to_summarize = [
    'age', 'hhsize', 'male',
    'k2110_2_wave7', 'cons_tot_wave7',
    'dedu1', 'dedu2', 'dedu3',
    'q2021_wave7_Temporarily laid-off (you expect to return to your previous workplace)',
    'q2021_wave7_Unemployed and actively looking for a job',
    'q2021_wave7_Unemployed, interested in having a job but not actively looking for a job',
    'q2021_wave7_Working full-time (self-employed or working for someone else)',
    'q2021_wave7_Working part-time (self-employed or working for someone else)',
    'cnt1','cnt2','cnt3','cnt4','cnt5','cnt6',
    'b3300_Owner-occupied property with mortgage outstanding',
    'b3300_Owner-occupied property without mortgage outstanding',
    'b3300_Rented house/flat'
]

# === Treatment dummies ===
treatments = ['treat1','treat2','treat3','treat4','treat5']

# Convert variables to numeric where possible
for v in vars_to_summarize + treatments:
    if v in df.columns:
        df[v] = pd.to_numeric(df[v], errors="coerce")

# === Build summary table ===
rows = []
for var in vars_to_summarize:
    if var not in df.columns:
        print(f"⚠️ Skipping {var} (not in dataset)")
        continue
    
    # Mean row
    row_mean = {"Variable": f"{var} (Mean)"}
    for t in treatments:
        row_mean[t] = df.loc[df[t]==1, var].mean()
    row_mean["All"] = df[var].mean()
    
    # Standard deviation row
    row_std = {"Variable": f"{var} (Std)"}
    for t in treatments:
        row_std[t] = df.loc[df[t]==1, var].std()
    row_std["All"] = df[var].std()
    
    # p-value from F-test across treatments
    try:
        X = df[treatments]
        y = df[var]
        model = sm.OLS(y, sm.add_constant(X), missing="drop").fit()
        f_test = model.f_test("treat1 = treat2 = treat3 = treat4 = treat5 = 0")
        pval = f_test.pvalue.item()
    except Exception:
        pval = None
    
    row_mean["p-value"] = pval
    row_std["p-value"] = None  # only show p-value on mean row
    
    # N row
    row_n = {"Variable": f"{var} (N)"}
    for t in treatments:
        row_n[t] = df.loc[df[t]==1, var].count()
    row_n["All"] = df[var].count()
    row_n["p-value"] = None
    
    rows.extend([row_mean, row_std, row_n])

# Assemble final table
table = pd.DataFrame(rows)

# Round numeric columns (except Variable and p-value)
numeric_cols = [c for c in table.columns if c not in ["Variable", "p-value"]]
table[numeric_cols] = table[numeric_cols].round(3)

# Convert N rows to integers
n_rows = table['Variable'].str.endswith("(N)")
table.loc[n_rows, numeric_cols] = table.loc[n_rows, numeric_cols].astype('Int64')

# === Save output ===
outpath = r"C:\Users\Zach\PycharmProjects\AppliedML\TreatmentBalanceTable.csv"
table.to_csv(outpath, index=False, float_format="%.3f")

print("✅ Saved summary table to:", outpath)


C:\Users\Zach\AppData\Local\Temp\ipykernel_19828\3191903027.py:5: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,651,652,653,654,655,656,657,658,659,660,672,683,697,698,699,705,706,707) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r"C:\Users\

✅ Saved summary table to: C:\Users\Zach\PycharmProjects\AppliedML\TreatmentBalanceTable.csv


C:\Users\Zach\anaconda3\Lib\site-packages\statsmodels\base\model.py:1888: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 5, but rank is 1
  warnings.warn('covariance of constraints does not have full '
C:\Users\Zach\anaconda3\Lib\site-packages\statsmodels\base\model.py:1888: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 5, but rank is 1
  warnings.warn('covariance of constraints does not have full '
C:\Users\Zach\anaconda3\Lib\site-packages\statsmodels\base\model.py:1888: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 5, but rank is 1
  warnings.warn('covariance of constraints does not have full '
